# IMPORTS

In [1]:
import os
from collections import defaultdict
import nltk
import re
from pathlib import Path
import random
import numpy as np
from collections import defaultdict
import matplotlib as plt
import math
import pickle
#nltk.download('punkt')

In [2]:
path_datasets = "../datasets/Data/"
path_vlachos  = "Vlachos/data_levy/"
path_levy     = "Levy/relation_splits/"
path_ours     = path_datasets + "Ours/"

vlachos_train = path_datasets + path_vlachos + "train.txt"
vlachos_train_oneshot = path_datasets + path_vlachos + "train_oneshot.txt"
vlachos_dev = path_datasets + path_vlachos + "dev.txt"
vlachos_eval = path_datasets + path_vlachos + "eval.txt"

vlachos_datasets_names = ['train', 'oneshot', 'dev', 'eval']
vlachos_datasets_names_id_map = {'train': 0, 'oneshot': 1, 'dev': 2, 'eval': 3}
vlachos_datasets_paths = [vlachos_train, vlachos_train_oneshot, vlachos_dev, vlachos_eval]

levy_datasets_names = [dataset_type + str(num) for dataset_type in ['train.', 'dev.', 'test.'] for num in range(0, 10)]
levy_datasets_names_id_map = {'train': 0, 'oneshot': 1, 'dev': 2, 'eval': 3}
levy_datasets_paths = [path_datasets + path_levy + dataset_name for dataset_name in levy_datasets_names]

levy_full_datasets_names = ['positive_examples', 'negative_examples']
levy_full_datasets_names_id_map = {'positive': 0, 'negative': 1}
levy_full_datasets_paths = [path_datasets + 'Levy/' + dataset_name for dataset_name in levy_full_datasets_names]

path_relation_names = path_ours + "relation_names.txt"
path_relation_descriptions = path_ours + "relations_descriptions/"# + each relation's name


glove_embeddings_general_path = path_datasets + "GloVe/glove."

### Notes

From service_entry (in the Levy by_relation/positive and in Vlachos equivalent entry in oneshot) manually reduced ENTITY_0 from plural to singular in the following entry:

"Including the single prototype, 141 ENTITY_0s were built from November 1943 to February ENTITY_1, entering service in ENTITY_1.	Flakpanzer 38(t)	1944" making it:

"Including the single prototype, 141 ENTITY_0 were built from November 1943 to February ENTITY_1, entering service in ENTITY_1.	Flakpanzer 38(t)	1944"

----------------------------------------------------------------

How to deal with plurals, like the example above, but that weren't properly mapped (in either Levy's or Vlachos') to start with?

----------------------

From cast_member (in the Levy by_relation/positive) corrected the following:

ENTITY_0 is a 1994 Hong Kong comedy film directed by Jacky Pang and starring ENTITY_1 Chiu-Wai, ENTITY_1ENTITY_3, ENTITY_4, ENTITY_5 and Charine Chan.	Always Be the Winners	Tony Leung	Tony Leung Ka-fai	Eric Tsang	Sandra Ng	Ekin Cheng

to

ENTITY_0 is a 1994 Hong Kong comedy film directed by Jacky Pang and starring ENTITY_1, ENTITY_2, ENTITY_3, ENTITY_4, ENTITY_5 and Charine Chan.	Always Be the Winners	Tony Leung Chiu-Wai	Tony Leung Ka-fai	Eric Tsang	Sandra Ng	Ekin Cheng

Also corrected:

ENTITY_0, also known as He Ain't Heavy, He's My Brother (新難兄難弟), is a 1993 Hong Kong comedy-drama film directed by Peter Chan and Lee Chi-ngai and starring ENTITY_1ENTITY_2 Chiu-wai, ENTITY_3, ENTITY_4 and ENTITY_5.	He Ain't Heavy, He's My Father	Tony Leung Ka-fai	Tony Leung	Carina Lau	Anita Yuen	Lawrence Cheng

to

ENTITY_0, also known as He Ain't Heavy, He's My Brother (新難兄難弟), is a 1993 Hong Kong comedy-drama film directed by Peter Chan and Lee Chi-ngai and starring ENTITY_1, ENTITY_2, ENTITY_3, ENTITY_4 and ENTITY_5.	He Ain't Heavy, He's My Father	Tony Leung Ka-fai	Tony Leung Chiu-Wai	Carina Lau	Anita Yuen	Lawrence Cheng

-----------------

From country (in the Levy by_relation/positive) corrected the following:

In the ENTITY_1the waters between the Florida Islands and Taivu Point on the northeast of Guadalcanal are divided by reefs into (from north to south) ENTITY_0, Sealark Channel, and Lengo Channel.	Nggela Channel	Solomon Islands	Solomon Islands

to

In the ENTITY_1, the waters between the Florida Islands and Taivu Point on the northeast of Guadalcanal are divided by reefs into (from north to south) ENTITY_0, Sealark Channel and Lengo Channel.	Nggela Channel	Solomon Islands

(also removed a similar entry (just had an extra comma))


--------------------

From country of origin (in the Levy by_relation/positive) corrected the following:

On October 9, 2014, Hulu ENTITY_1 announced that some TV shows were made available for streaming including ENTITY_03.	NogiBingo!	Japan

to

On October 9, 2014, Hulu ENTITY_1 announced that some TV shows were made available for streaming including ENTITY_0.	NogiBingo!3	Japan

-----------------

From father (in the Levy by_relation/positive) corrected the following:

ENTITY_0Arabic: بادلاي بن سعد الدين‎) (also known as Sihab ad-Din Ahmad Badlay, Arwe Badlay - \`\`Badlay the Beast'') (died 1445) was a Sultan of the Sultanate of Adal and a son of ENTITY_1.	Badlay ibn Sa'ad ad-Din	Sa'ad ad-Din II

to

ENTITY_0 (Arabic: بادلاي بن سعد الدين‎) (also known as Sihab ad-Din Ahmad Badlay, Arwe Badlay - \`\`Badlay the Beast'') (died 1445) was a Sultan of the Sultanate of Adal and a son of ENTITY_1.	Badlay ibn Sa'ad ad-Din II	Sa'ad ad-Din II

--------------

From home venue (in the Levy by_relation/positive) corrected the following:

ENTITY_0 play their home games at the ENTITY_1in the city of Tacna.	Coronel Bolognesi	Estadio Jorge Basadre	Estadio Jorge Basadre

to

ENTITY_0 play their home games at the ENTITY_1 in the city of Tacna.	Coronel Bolognesi	Estadio Jorge Basadre

------

From located in the administrative territorial entity (in the Levy by_relation/positive) corrected the following:

ENTITY_0 is a department in ENTITY_1Argentina.	Leales Department	Tucumán Province	Tucumán Province

to

ENTITY_0 is a department in ENTITY_1, Argentina.	Leales Department	Tucumán Province

Also corrected:

ENTITY_0 is an American public high school in ENTITY_1, California, and is part of the ENTITY_1SMUHSD).	Aragon High School	San Mateo	San Mateo Union High School District

to

ENTITY_0 is an American public high school in ENTITY_1, California, and is part of the ENTITY_2 (SMUHSD).	Aragon High School	San Mateo	San Mateo Union High School District

Also corrected:

ENTITY_0 (Ukrainian: Первомайський район, Russian: Первомайский район, Crimean Tatar: Curçı rayonı) is one of the 25 regions of the ENTITY_1territory recognized by a majority of countries as part of Ukraine and incorporated by Russia as the ENTITY_2.	Pervomaiske Raion	Autonomous Republic of Crimea	Republic of Crimea

to

ENTITY_0 (Ukrainian: Первомайський район, Russian: Первомайский район, Crimean Tatar: Curçı rayonı) is one of the 25 regions of the ENTITY_1 territory recognized by a majority of countries as part of Ukraine and incorporated by Russia as the ENTITY_2.	Pervomaiske Raion	Autonomous Republic of Crimea	Republic of Crimea

Also corrected:

ENTITY_0 Pierwsze (sarˈbit͡sɛ ˈpjɛrfʂɛ) is a village in the administrative district of ENTITY_1within Kielce County, Świętokrzyskie Voivodeship, in south-central Poland.	Sarbice	Gmina Łopuszno	Gmina Łopuszno

to

ENTITY_0 (sarˈbit͡sɛ ˈpjɛrfʂɛ) is a village in the administrative district of ENTITY_1, within Kielce County, Świętokrzyskie Voivodeship, in south-central Poland.	Sarbice Pierwsze	Gmina Łopuszno

Also corrected:

ENTITY_0 (ˈvɔla ɔsɔˈviɲska) is a village in the administrative district of ENTITY_1within Radzyń Podlaski County, Lublin Voivodeship, in eastern Poland.	Wola Osowińska	Gmina Borki	Gmina Borki

to

ENTITY_0 (ˈvɔla ɔsɔˈviɲska) is a village in the administrative district of ENTITY_1, within Radzyń Podlaski County, Lublin Voivodeship, in eastern Poland.	Wola Osowińska	Gmina Borki

Also corrected:

ENTITY_0 (Ukrainian: Нижньогірський район, Russian: Нижнегорский район, Crimean Tatar: Seyitler rayonı) is one of the 25 regions of the ENTITY_1territory recognized by a majority of countries as part of Ukraine and incorporated by Russia as the ENTITY_2.	Nyzhnohirskyi Raion	Autonomous Republic of Crimea	Republic of Crimea

to

ENTITY_0 (Ukrainian: Нижньогірський район, Russian: Нижнегорский район, Crimean Tatar: Seyitler rayonı) is one of the 25 regions of the ENTITY_1 territory, recognized by a majority of countries as part of Ukraine and incorporated by Russia as the ENTITY_2.	Nyzhnohirskyi Raion	Autonomous Republic of Crimea	Republic of Crimea

Also corrected:

ENTITY_0 (Ukrainian: Красноперекопський район, Russian: Красноперекопский район, Crimean Tatar: Krasnoperekopsk rayonı) is one of the 25 regions of the ENTITY_1territory recognized by a majority of countries as part of Ukraine and incorporated by Russia as the ENTITY_2.	Krasnoperekopsk Raion	Autonomous Republic of Crimea	Republic of Crimea

to

ENTITY_0 (Ukrainian: Красноперекопський район, Russian: Красноперекопский район, Crimean Tatar: Krasnoperekopsk rayonı) is one of the 25 regions of the ENTITY_1 territory, recognized by a majority of countries as part of Ukraine and incorporated by Russia as the ENTITY_2.	Krasnoperekopsk Raion	Autonomous Republic of Crimea	Republic of Crimea

Also corrected:

ENTITY_0 (Ukrainian: Роздольненський район, Russian: Раздольненский район, Crimean Tatar: Aqşeyh rayonı) is one of the 25 regions of the ENTITY_1territory recognized by a majority of countries as part of Ukraine and incorporated by Russia as the ENTITY_2.	Rozdolne Raion	Autonomous Republic of Crimea	Republic of Crimea

to

ENTITY_0 (Ukrainian: Роздольненський район, Russian: Раздольненский район, Crimean Tatar: Aqşeyh rayonı) is one of the 25 regions of the ENTITY_1 territory, recognized by a majority of countries as part of Ukraine and incorporated by Russia as the ENTITY_2.	Rozdolne Raion	Autonomous Republic of Crimea	Republic of Crimea

Also corrected:

ENTITY_0 (Ukrainian: Сакський район, Russian: Сакский район, Crimean Tatar: Saq rayonı) is one of the 25 regions of the ENTITY_1territory recognized by a majority of countries as part of Ukraine and incorporated by Russia as the ENTITY_2.	Saky Raion	Autonomous Republic of Crimea	Republic of Crimea

to

ENTITY_0 (Ukrainian: Сакський район, Russian: Сакский район, Crimean Tatar: Saq rayonı) is one of the 25 regions of the ENTITY_1 territory, recognized by a majority of countries as part of Ukraine and incorporated by Russia as the ENTITY_2.	Saky Raion	Autonomous Republic of Crimea	Republic of Crimea

Also corrected:

ENTITY_0 is a public artwork by American artist Tony Smith, located in the Marsh & McLennan Companies (MMC) Plaza at 1166 Avenue of the Americas in ENTITY_1ENTITY_2.	Throwback (1/3)	New York City	New York

to

ENTITY_0 is a public artwork by American artist Tony Smith, located in the Marsh & McLennan Companies (MMC) Plaza at 1166 Avenue of the Americas in ENTITY_1, ENTITY_2.	Throwback (1/3)	New York City	New York

Also corrected:

ENTITY_0, is located in Hoboken, Hudson County, ENTITY_1United States.	Engine Company No. 3	New Jersey	New Jersey

to

ENTITY_0, is located in Hoboken, Hudson County, ENTITY_1, United States.	Engine Company No. 3	New Jersey

Also corrected:

ENTITY_0 is the name of a skyscraper on Sowwah island in ENTITY_1the capital of the United Arab Emirates.	Sowwah Square Tower 2	Abu Dhabi	Abu Dhabi

to

ENTITY_0 is the name of a skyscraper on Sowwah island in ENTITY_1, the capital of the United Arab Emirates.	Sowwah Square Tower 2	Abu Dhabi

------------------

From occupation (in the Levy by_relation/positive) corrected the following:

ENTITY_0 (born 11 September 1984) is a British ENTITY_1and former Scottish national indoor ENTITY_2.	Shauna Mullin	beach volleyball player	volleyball player

to

ENTITY_0 (born 11 September 1984) is a British ENTITY_1 and former Scottish national indoor ENTITY_2.	Shauna Mullin	beach volleyball player	volleyball player

-------

From place of birth (in the Levy by_relation/positive) corrected the following:

ENTITY_0 De Avila (Born in ENTITY_1Brazil, May 11, 1984) is a Brazilian footballer.	Fernando Oliveira	Porto Alegre	Porto Alegre

to

ENTITY_0 De Avila (Born in ENTITY_1, Brazil, May 11, 1984) is a Brazilian footballer.	Fernando Oliveira	Porto Alegre

----

From vessel class (in the Levy by_relation/positive) corrected the following:

ENTITY_0/(DMS-2) was a ENTITY_1 in the United States Navy.	USS Lamberton (DD-119)	Wickes-class destroyer

to

ENTITY_0 was a ENTITY_1 in the United States Navy.	USS Lamberton (DD-119)/(DMS-2)	Wickes-class destroyer



-------------
----------------
---------------
-------------
-------------

Noticed errors:

creator	Marvel's Agents of S.H.I.E.L.D., or simply Agents of S.H.I.E.L.D., is an American television series created for ABC by OBJECT_ENTITY, OBJECT_ENTITY, and OBJECT_ENTITY, based on the Marvel Comics organization S.H.I.E.L.D. (Strategic Homeland Intervention, Enforcement, and Logistics Division), a fictional peacekeeping and spy agency in a world of superheroes.

As originally developed by screenwriter ENTITY_1, Klingons were swarthy humanoids characterized mainly by prideful ruthlessness and brutality.

## For Vlachos' relations descriptions, corrected the following:

### In architect:

OBJECT_ENTITY was responsible for the des ign of SUBJECT_ENTITY

to

OBJECT_ENTITY was responsible for the design of SUBJECT_ENTITY




OBJECT_ENTITY was the designor of SUBJECT_ENTITY

to

OBJECT_ENTITY was the designer of SUBJECT_ENTITY - AND REMOVED IT SINCE IT'S ALSO ON 'designer' AND 'designer' HAS FEWER RELATION DESCRIPTIONS

------------

In author:

OBJECT_ENTITY is the SUBJECT_ENTITY by (???? what the hell)



OBJECT_ENTITY was SUBJECT_ENTITY written by

to

OBJECT_ENTITY was written by SUBJECT_ENTITY (because what the hell....)



OBJECT_ENTITY was the auther of SUBJECT_ENTITY

to

OBJECT_ENTITY was the author of SUBJECT_ENTITY


FINISHED WITH on 'conflict', which has a lot of weirds descriptions....

# DATA PATHS AND CONSTANT VARIABLES

In [3]:
print(os.getcwd())

D:\Academic & Professional\Academic\Master Thesis AI\Master-Thesis\Code\Notebooks


In [8]:
path_data = path_datasets + path_ours + "sentences_only_test/normal/train.txt"

seen_token = defaultdict(lambda: False)
with open(path_data, 'r', encoding='utf-8') as f:
    for l_num, line in enumerate(f):
        if (l_num % 250 == 0):
            print('\r' + str(l_num), end="")
        split_line = line.rstrip('\r\n').split(' ')
        for token in split_line:
            if (not seen_token[token]):
                seen_token[token] = True

print("\nTotal number of unique tokens: " + str(sum(seen_token.values())))

6500
Total number of unique tokens: 28944


In [5]:
file_name_positive = path_datasets + 'Levy/by_relation/positive/author.txt'

num_sentences = 10000
destination_file = path_datasets + 'Ours/elmo_hdf5_test.txt'

data = []

with open(file_name_positive, "r", encoding='utf-8') as f:
    for l_num, line in enumerate(f):
        if (l_num == num_sentences):
            break
        data.append(line.rstrip('\r\n').split('\t')[0])

with open(destination_file, 'w+', encoding='utf-8') as f:
    for data_instance in data:
        f.write(data_instance + '\n')

# READ DATA

In [4]:
relation_counts_train = defaultdict(lambda: 0)
relation_counts_train_oneshot = defaultdict(lambda: 0)
relation_counts_dev  = defaultdict(lambda: 0)
relation_counts_eval = defaultdict(lambda: 0)
realtion_counts_all  = defaultdict(lambda: 0)

#TODO: Tokenize sentences


with open(vlachos_train, "r", encoding='utf-8') as f:
    for line in f:
        line_temp = line.rstrip("\r\n").split("\t")
        relation_counts_train[line_temp[1]] += 1
        realtion_counts_all[line_temp[1]] += 1

print(len(relation_counts_train))
print(relation_counts_train)

print("\n\n")



with open(vlachos_train_oneshot, "r", encoding='utf-8') as f:
    for line in f:
        line_temp = line.rstrip("\r\n").split("\t")
        relation_counts_train_oneshot[line_temp[1]] += 1
        realtion_counts_all[line_temp[1]] += 1

print(len(relation_counts_train_oneshot))
print(relation_counts_train_oneshot)

print("\n\n")



with open(vlachos_dev, "r", encoding='utf-8') as f:
    for line in f:
        line_temp = line.rstrip("\r\n").split("\t")
        relation_counts_dev[line_temp[1]] += 1
        realtion_counts_all[line_temp[1]] += 1

print(len(relation_counts_dev))
print(relation_counts_dev)

print("\n\n")



with open(vlachos_eval, "r", encoding='utf-8') as f:
    for line in f:
        line_temp = line.rstrip("\r\n").split("\t")
        relation_counts_eval[line_temp[1]] += 1
        realtion_counts_all[line_temp[1]] += 1

print(len(relation_counts_eval))
print(relation_counts_eval)

24
defaultdict(<function <lambda> at 0x00000185D75329D8>, {'license': 448, 'military branch': 21934, 'founder': 23142, 'chairperson': 390, 'end time': 1732, 'continent': 56314, 'spouse': 4016, 'constellation': 12208, 'country of origin': 26880, 'location of formation': 4872, 'head of government': 1048, 'based on': 2052, 'vessel class': 912, 'IUCN conservation status': 4168, 'located next to body of water': 1620, 'airline hub': 5616, 'brother': 4514, 'date of death': 63676, 'country of citizenship': 57218, 'start time': 1366, 'programming language': 3848, 'record label': 23520, 'film editor': 10848, 'date of official opening': 3556})



72
defaultdict(<function <lambda> at 0x00000185D7532950>, {'position played on team / speciality': 17028, 'parent company': 1842, 'member of sports team': 14404, 'mouth of the watercourse': 8934, 'sport': 13726, 'drafted by': 174, 'taxon rank': 3378, 'designer': 5302, 'place of birth': 28356, 'point in time': 1334, 'voice type': 5874, 'architect': 10332,

In [51]:
print(len(realtion_counts_all))
print(realtion_counts_all)

120
defaultdict(<function <lambda> at 0x0000025482D728C8>, {'license': 448, 'military branch': 21934, 'founder': 23142, 'chairperson': 390, 'end time': 1732, 'continent': 56314, 'spouse': 4016, 'constellation': 12208, 'country of origin': 26880, 'location of formation': 4872, 'head of government': 1048, 'based on': 2052, 'vessel class': 912, 'IUCN conservation status': 4168, 'located next to body of water': 1620, 'airline hub': 5616, 'brother': 4514, 'date of death': 63676, 'country of citizenship': 57218, 'start time': 1366, 'programming language': 3848, 'record label': 23520, 'film editor': 10848, 'date of official opening': 3556, 'position played on team / speciality': 34056, 'parent company': 3684, 'member of sports team': 28808, 'mouth of the watercourse': 17866, 'sport': 27450, 'drafted by': 348, 'taxon rank': 6756, 'designer': 10602, 'place of birth': 56712, 'point in time': 2668, 'voice type': 11748, 'architect': 20664, 'chromosome': 3820, 'military rank': 340, 'illustrator': 2

In [46]:
train_not_in_dev = []

for relation in relation_counts_train:
    if relation not in relation_counts_dev:
        train_not_in_dev.append(relation)

print(len(train_not_in_dev))
print(train_not_in_dev)

24
['license', 'military branch', 'founder', 'chairperson', 'end time', 'continent', 'spouse', 'constellation', 'country of origin', 'location of formation', 'head of government', 'based on', 'vessel class', 'IUCN conservation status', 'located next to body of water', 'airline hub', 'brother', 'date of death', 'country of citizenship', 'start time', 'programming language', 'record label', 'film editor', 'date of official opening']


In [47]:
train_not_in_eval = []

for relation in relation_counts_train:
    if relation not in relation_counts_eval:
        train_not_in_eval.append(relation)

print(len(train_not_in_eval))
print(train_not_in_eval)

24
['license', 'military branch', 'founder', 'chairperson', 'end time', 'continent', 'spouse', 'constellation', 'country of origin', 'location of formation', 'head of government', 'based on', 'vessel class', 'IUCN conservation status', 'located next to body of water', 'airline hub', 'brother', 'date of death', 'country of citizenship', 'start time', 'programming language', 'record label', 'film editor', 'date of official opening']


In [52]:
dev_not_in_eval = []

for relation in relation_counts_dev:
    if relation not in relation_counts_eval:
        dev_not_in_eval.append(relation)

print(len(dev_not_in_eval))
print(dev_not_in_eval)

24
['sex or gender', 'medical condition', 'headquarters location', 'employer', 'located in the administrative territorial entity', 'editor', 'parent taxon', 'date of birth', 'home venue', 'publication date', 'industry', 'discoverer or inventor', 'found in taxon', 'occupant', 'crosses', 'distributor', 'award received', 'cast member', 'architectural style', 'sister', 'product', 'conflict', 'educated at', 'author']


In [53]:
oneshot_not_in_dev = []

for relation in relation_counts_train_oneshot:
    if relation not in relation_counts_dev:
        oneshot_not_in_dev.append(relation)

print(len(oneshot_not_in_dev))
print(oneshot_not_in_dev)

72
['position played on team / speciality', 'parent company', 'member of sports team', 'mouth of the watercourse', 'sport', 'drafted by', 'taxon rank', 'designer', 'place of birth', 'point in time', 'voice type', 'architect', 'chromosome', 'military rank', 'illustrator', 'father', 'time of discovery', 'standards body', 'place of death', 'instrument', 'manufacturer', 'creator', 'performer', 'operating system', 'conferred by', 'production company', 'league', 'participant of', 'country', 'member of political party', 'licensed to broadcast to', 'noble family', 'named after', 'nominated for', 'child', 'present in work', 'time of spacecraft launch', 'stock exchange', 'service entry', 'narrative location', 'native language', 'connecting line', 'screenwriter', 'series', 'site of astronomical discovery', 'place of burial', 'located on astronomical body', 'instrumentation', 'lyrics by', 'language of work or name', 'languages spoken or written', 'publisher', 'dissolved or abolished', 'collection'

In [54]:
oneshot_not_in_eval = []

for relation in relation_counts_train_oneshot:
    if relation not in relation_counts_eval:
        oneshot_not_in_eval.append(relation)

print(len(oneshot_not_in_eval))
print(oneshot_not_in_eval)

1
['standards body']


In [55]:
train_not_in_oneshot = []

for relation in relation_counts_train:
    if relation not in relation_counts_train_oneshot:
        train_not_in_oneshot.append(relation)

print(len(train_not_in_oneshot))
print(train_not_in_oneshot)

24
['license', 'military branch', 'founder', 'chairperson', 'end time', 'continent', 'spouse', 'constellation', 'country of origin', 'location of formation', 'head of government', 'based on', 'vessel class', 'IUCN conservation status', 'located next to body of water', 'airline hub', 'brother', 'date of death', 'country of citizenship', 'start time', 'programming language', 'record label', 'film editor', 'date of official opening']


### Check for overlap percentage between the train_one_shot and the test split

In [2]:
def file_len(fname):
    with open(fname, "r", encoding='utf-8') as f:
        for i, l in enumerate(f):
            pass
    return i + 1

In [ ]:
file_length_one_shot = file_len(vlachos_train_oneshot)
file_length_test = file_len(vlachos_eval)

In [40]:
sentences_in_oneshot = {}
num_sentences_per_relation_in_oneshot = {}
with open(vlachos_train_oneshot, "r", encoding='utf-8') as f:
    for l_num, line in enumerate(f):
        if (l_num % min(int(file_length_one_shot/3), 250) == 0 or l_num + 1 == file_length_one_shot):
                print('\r' + str(l_num + 1) + '/' + str(file_length_one_shot), end="", flush=True)

        line_temp = line.rstrip("\r\n").split("\t")
        if (line_temp[-1] != "tNEGATIVE"):
            if (line_temp[1] not in sentences_in_oneshot):
                sentences_in_oneshot[line_temp[1]] = {}
                num_sentences_per_relation_in_oneshot[line_temp[1]] = 1
            else:
                num_sentences_per_relation_in_oneshot[line_temp[1]] += 1

            split_sentence = line_temp[2].split(' ')
            if (split_sentence[min(4, len(split_sentence) - 1)] not in sentences_in_oneshot[line_temp[1]]):
                sentences_in_oneshot[line_temp[1]][split_sentence[min(4, len(split_sentence) - 1)]] = ['\t'.join(line_temp[2:3])]#[line_temp[2]]#['\t'.join(line_temp[1:])]
            else:
                sentences_in_oneshot[line_temp[1]][split_sentence[min(4, len(split_sentence) - 1)]].append('\t'.join(line_temp[2:3]))#line_temp[2])#'\t'.join(line_temp[1:]))

print()

sentences_in_test = {}
num_sentences_per_relation_in_test = {}
with open(vlachos_eval, "r", encoding='utf-8') as f:
    for l_num, line in enumerate(f):
        if (l_num % min(int(file_length_test/3), 250) == 0 or l_num + 1 == file_length_test):
                print('\r' + str(l_num + 1) + '/' + str(file_length_test), end="", flush=True)

        line_temp = line.rstrip("\r\n").split("\t")
        if (line_temp[-1] != "tNEGATIVE"):
            if (line_temp[1] not in sentences_in_test):
                sentences_in_test[line_temp[1]] = {}
                num_sentences_per_relation_in_test[line_temp[1]] = 1
            else:
                num_sentences_per_relation_in_test[line_temp[1]] += 1

            split_sentence = line_temp[2].split(' ')
            if (split_sentence[min(4, len(split_sentence) - 1)] not in sentences_in_test[line_temp[1]]):
                sentences_in_test[line_temp[1]][split_sentence[min(4, len(split_sentence) - 1)]] = ['\t'.join(line_temp[2:3])]#[line_temp[2]]#['\t'.join(line_temp[1:])]
            else:
                sentences_in_test[line_temp[1]][split_sentence[min(4, len(split_sentence) - 1)]].append('\t'.join(line_temp[2:3]))#line_temp[2])#'\t'.join(line_temp[1:]))

580204/580204
580148/580148

In [41]:
num_sentences_of_oneshot_in_test_per_relation = {}
l_num = 0
for relation in sentences_in_oneshot:
    if (relation not in num_sentences_of_oneshot_in_test_per_relation):
            num_sentences_of_oneshot_in_test_per_relation[relation] = 0

    for key_word in sentences_in_oneshot[relation]:
        for line in sentences_in_oneshot[relation][key_word]:
            l_num += 1
            if (l_num % min(int(file_length_one_shot/3), 250) == 0 or l_num + 1 == file_length_one_shot):
                print('\r' + str(l_num + 1) + '/' + str(file_length_one_shot), end="", flush=True)

            if (relation in sentences_in_test):
                if (key_word in sentences_in_test[relation]):
                    if (line in sentences_in_test[relation][key_word]):
                        num_sentences_of_oneshot_in_test_per_relation[relation] += 1

print()

num_sentences_of_test_in_oneshot_per_relation = {}
l_num = 0
for relation in sentences_in_test:
    if (relation not in num_sentences_of_test_in_oneshot_per_relation):
        num_sentences_of_test_in_oneshot_per_relation[relation] = 0

    for key_word in sentences_in_test[relation]:
        for line in sentences_in_test[relation][key_word]:
            l_num += 1
            if (l_num % min(int(file_length_test/3), 250) == 0 or l_num + 1 == file_length_test):
                print('\r' + str(l_num + 1) + '/' + str(file_length_test), end="", flush=True)

            if (relation in sentences_in_oneshot):
                if (key_word in sentences_in_oneshot[relation]):
                    if (line in sentences_in_oneshot[relation][key_word]):
                        num_sentences_of_test_in_oneshot_per_relation[relation] += 1

580204/580204
580148/580148

### Both Labels

In [13]:
def count_totals(temp_dict):
    counter = 0
    for key in temp_dict:
        counter += temp_dict[key]
    return counter

In [17]:
# '\t'.join(line_temp[1:]) <=> '\t'.join(line_temp[1:5]) --- relation, x, y, label
print(any(num_sentences_of_oneshot_in_test_per_relation[relation] != 0 for relation in num_sentences_of_oneshot_in_test_per_relation))
temp = {relation: num_sentences_of_oneshot_in_test_per_relation[relation] for relation in num_sentences_of_oneshot_in_test_per_relation if num_sentences_of_oneshot_in_test_per_relation[relation] != 0}
print(len(temp))
print(temp)
print(count_totals(temp))
print()

print(any(num_sentences_of_test_in_oneshot_per_relation[relation] != 0 for relation in num_sentences_of_test_in_oneshot_per_relation))
temp = {relation: num_sentences_of_test_in_oneshot_per_relation[relation] for relation in num_sentences_of_test_in_oneshot_per_relation if num_sentences_of_test_in_oneshot_per_relation[relation] != 0}
print(len(temp))
print(temp)
print(count_totals(temp))

False
0
{}
0

False
0
{}
0


In [21]:
# '\t'.join(line_temp[1:4]) --- relation, x, y
print(any(num_sentences_of_oneshot_in_test_per_relation[relation] != 0 for relation in num_sentences_of_oneshot_in_test_per_relation))
temp = {relation: num_sentences_of_oneshot_in_test_per_relation[relation] for relation in num_sentences_of_oneshot_in_test_per_relation if num_sentences_of_oneshot_in_test_per_relation[relation] != 0}
print(len(temp))
print(temp)
print(count_totals(temp))
print()

print(any(num_sentences_of_test_in_oneshot_per_relation[relation] != 0 for relation in num_sentences_of_test_in_oneshot_per_relation))
temp = {relation: num_sentences_of_test_in_oneshot_per_relation[relation] for relation in num_sentences_of_test_in_oneshot_per_relation if num_sentences_of_test_in_oneshot_per_relation[relation] != 0}
print(len(temp))
print(temp)
print(count_totals(temp))

True
39
{'position played on team / speciality': 1, 'parent company': 3, 'member of sports team': 13, 'mouth of the watercourse': 10, 'taxon rank': 1, 'designer': 3, 'point in time': 3, 'military rank': 1, 'father': 11, 'place of death': 4, 'instrument': 3, 'manufacturer': 20, 'creator': 8, 'performer': 6, 'production company': 7, 'league': 3, 'participant of': 2, 'country': 1, 'noble family': 1, 'named after': 2, 'child': 11, 'present in work': 12, 'stock exchange': 4, 'narrative location': 4, 'screenwriter': 26, 'series': 4, 'lyrics by': 50, 'language of work or name': 9, 'languages spoken or written': 3, 'publisher': 9, 'dissolved or abolished': 3, 'director': 4, 'mother': 6, 'developer': 16, 'residence': 4, 'occupation': 3, 'characters': 1, 'position held': 1, 'original network': 4}
277

True
39
{'position played on team / speciality': 1, 'parent company': 3, 'member of sports team': 13, 'mouth of the watercourse': 10, 'taxon rank': 1, 'designer': 3, 'point in time': 3, 'military r

In [24]:
# '\t'.join(line_temp[1:3]) --- relation, x
print(any(num_sentences_of_oneshot_in_test_per_relation[relation] != 0 for relation in num_sentences_of_oneshot_in_test_per_relation))
temp = {relation: num_sentences_of_oneshot_in_test_per_relation[relation] for relation in num_sentences_of_oneshot_in_test_per_relation if num_sentences_of_oneshot_in_test_per_relation[relation] != 0}
print(len(temp))
print(temp)
print(count_totals(temp))
print()

print(any(num_sentences_of_test_in_oneshot_per_relation[relation] != 0 for relation in num_sentences_of_test_in_oneshot_per_relation))
temp = {relation: num_sentences_of_test_in_oneshot_per_relation[relation] for relation in num_sentences_of_test_in_oneshot_per_relation if num_sentences_of_test_in_oneshot_per_relation[relation] != 0}
print(len(temp))
print(temp)
print(count_totals(temp))

True
67
{'position played on team / speciality': 15625, 'parent company': 1019, 'member of sports team': 12780, 'mouth of the watercourse': 8125, 'sport': 11714, 'drafted by': 94, 'taxon rank': 1180, 'designer': 4603, 'place of birth': 24499, 'point in time': 994, 'voice type': 4793, 'architect': 10241, 'chromosome': 1446, 'military rank': 76, 'illustrator': 1043, 'father': 20406, 'time of discovery': 2517, 'place of death': 13743, 'instrument': 11931, 'manufacturer': 13845, 'creator': 9792, 'performer': 19439, 'production company': 3708, 'league': 1507, 'participant of': 5817, 'country': 21715, 'member of political party': 6552, 'licensed to broadcast to': 211, 'noble family': 752, 'named after': 3936, 'nominated for': 67, 'child': 1082, 'present in work': 5734, 'time of spacecraft launch': 3416, 'stock exchange': 4761, 'service entry': 2123, 'narrative location': 716, 'native language': 9902, 'connecting line': 595, 'screenwriter': 16887, 'series': 18370, 'site of astronomical discov

In [27]:
# '\t'.join(line_temp[2:4]) --- x, y
print(any(num_sentences_of_oneshot_in_test_per_relation[relation] != 0 for relation in num_sentences_of_oneshot_in_test_per_relation))
temp = {relation: num_sentences_of_oneshot_in_test_per_relation[relation] for relation in num_sentences_of_oneshot_in_test_per_relation if num_sentences_of_oneshot_in_test_per_relation[relation] != 0}
print(len(temp))
print(temp)
print(count_totals(temp))
print()

print(any(num_sentences_of_test_in_oneshot_per_relation[relation] != 0 for relation in num_sentences_of_test_in_oneshot_per_relation))
temp = {relation: num_sentences_of_test_in_oneshot_per_relation[relation] for relation in num_sentences_of_test_in_oneshot_per_relation if num_sentences_of_test_in_oneshot_per_relation[relation] != 0}
print(len(temp))
print(temp)
print(count_totals(temp))

True
39
{'position played on team / speciality': 1, 'parent company': 3, 'member of sports team': 13, 'mouth of the watercourse': 10, 'taxon rank': 1, 'designer': 3, 'point in time': 3, 'military rank': 1, 'father': 11, 'place of death': 4, 'instrument': 3, 'manufacturer': 20, 'creator': 8, 'performer': 6, 'production company': 7, 'league': 3, 'participant of': 2, 'country': 1, 'noble family': 1, 'named after': 2, 'child': 11, 'present in work': 12, 'stock exchange': 4, 'narrative location': 4, 'screenwriter': 26, 'series': 4, 'lyrics by': 50, 'language of work or name': 9, 'languages spoken or written': 3, 'publisher': 9, 'dissolved or abolished': 3, 'director': 4, 'mother': 6, 'developer': 16, 'residence': 4, 'occupation': 3, 'characters': 1, 'position held': 1, 'original network': 4}
277

True
39
{'position played on team / speciality': 1, 'parent company': 3, 'member of sports team': 13, 'mouth of the watercourse': 10, 'taxon rank': 1, 'designer': 3, 'point in time': 3, 'military r

In [42]:
# '\t'.join(line_temp[2:3]) --- x
print(any(num_sentences_of_oneshot_in_test_per_relation[relation] != 0 for relation in num_sentences_of_oneshot_in_test_per_relation))
temp = {relation: num_sentences_of_oneshot_in_test_per_relation[relation] for relation in num_sentences_of_oneshot_in_test_per_relation if num_sentences_of_oneshot_in_test_per_relation[relation] != 0}
print(len(temp))
print(temp)
print(count_totals(temp))
print()

print(any(num_sentences_of_test_in_oneshot_per_relation[relation] != 0 for relation in num_sentences_of_test_in_oneshot_per_relation))
temp = {relation: num_sentences_of_test_in_oneshot_per_relation[relation] for relation in num_sentences_of_test_in_oneshot_per_relation if num_sentences_of_test_in_oneshot_per_relation[relation] != 0}
print(len(temp))
print(temp)
print(count_totals(temp))

True
67
{'position played on team / speciality': 15625, 'parent company': 1019, 'member of sports team': 12780, 'mouth of the watercourse': 8125, 'sport': 11714, 'drafted by': 94, 'taxon rank': 1180, 'designer': 4603, 'place of birth': 24499, 'point in time': 994, 'voice type': 4793, 'architect': 10241, 'chromosome': 1446, 'military rank': 76, 'illustrator': 1043, 'father': 20406, 'time of discovery': 2517, 'place of death': 13743, 'instrument': 11931, 'manufacturer': 13845, 'creator': 9792, 'performer': 19439, 'production company': 3708, 'league': 1507, 'participant of': 5817, 'country': 21715, 'member of political party': 6552, 'licensed to broadcast to': 211, 'noble family': 752, 'named after': 3936, 'nominated for': 67, 'child': 1082, 'present in work': 5734, 'time of spacecraft launch': 3416, 'stock exchange': 4761, 'service entry': 2123, 'narrative location': 716, 'native language': 9902, 'connecting line': 595, 'screenwriter': 16887, 'series': 18370, 'site of astronomical discov

### Removing the NEGATIVE label

In [30]:
# '\t'.join(line_temp[1:4]) --- relation, x, y
print(any(num_sentences_of_oneshot_in_test_per_relation[relation] != 0 for relation in num_sentences_of_oneshot_in_test_per_relation))
temp = {relation: num_sentences_of_oneshot_in_test_per_relation[relation] for relation in num_sentences_of_oneshot_in_test_per_relation if num_sentences_of_oneshot_in_test_per_relation[relation] != 0}
print(len(temp))
print(temp)
print(count_totals(temp))
print()

print(any(num_sentences_of_test_in_oneshot_per_relation[relation] != 0 for relation in num_sentences_of_test_in_oneshot_per_relation))
temp = {relation: num_sentences_of_test_in_oneshot_per_relation[relation] for relation in num_sentences_of_test_in_oneshot_per_relation if num_sentences_of_test_in_oneshot_per_relation[relation] != 0}
print(len(temp))
print(temp)
print(count_totals(temp))

False
0
{}
0

False
0
{}
0


In [33]:
# '\t'.join(line_temp[1:3]) --- relation, x
print(any(num_sentences_of_oneshot_in_test_per_relation[relation] != 0 for relation in num_sentences_of_oneshot_in_test_per_relation))
temp = {relation: num_sentences_of_oneshot_in_test_per_relation[relation] for relation in num_sentences_of_oneshot_in_test_per_relation if num_sentences_of_oneshot_in_test_per_relation[relation] != 0}
print(len(temp))
print(temp)
print(count_totals(temp))
print()

print(any(num_sentences_of_test_in_oneshot_per_relation[relation] != 0 for relation in num_sentences_of_test_in_oneshot_per_relation))
temp = {relation: num_sentences_of_test_in_oneshot_per_relation[relation] for relation in num_sentences_of_test_in_oneshot_per_relation if num_sentences_of_test_in_oneshot_per_relation[relation] != 0}
print(len(temp))
print(temp)
print(count_totals(temp))

True
66
{'position played on team / speciality': 7135, 'parent company': 711, 'member of sports team': 5663, 'mouth of the watercourse': 4270, 'sport': 5158, 'drafted by': 7, 'taxon rank': 435, 'designer': 2642, 'place of birth': 13426, 'point in time': 325, 'voice type': 2099, 'architect': 5166, 'chromosome': 955, 'military rank': 32, 'illustrator': 564, 'father': 10489, 'time of discovery': 1264, 'place of death': 7551, 'instrument': 5993, 'manufacturer': 7015, 'creator': 5261, 'performer': 12793, 'production company': 2255, 'league': 692, 'participant of': 2220, 'country': 13834, 'member of political party': 2672, 'licensed to broadcast to': 19, 'noble family': 345, 'named after': 2720, 'nominated for': 54, 'child': 745, 'present in work': 2776, 'time of spacecraft launch': 2030, 'stock exchange': 2539, 'service entry': 1123, 'narrative location': 536, 'native language': 4468, 'connecting line': 178, 'screenwriter': 9268, 'series': 10209, 'site of astronomical discovery': 271, 'plac

In [36]:
# '\t'.join(line_temp[2:4]) --- x, y
print(any(num_sentences_of_oneshot_in_test_per_relation[relation] != 0 for relation in num_sentences_of_oneshot_in_test_per_relation))
temp = {relation: num_sentences_of_oneshot_in_test_per_relation[relation] for relation in num_sentences_of_oneshot_in_test_per_relation if num_sentences_of_oneshot_in_test_per_relation[relation] != 0}
print(len(temp))
print(temp)
print(count_totals(temp))
print()

print(any(num_sentences_of_test_in_oneshot_per_relation[relation] != 0 for relation in num_sentences_of_test_in_oneshot_per_relation))
temp = {relation: num_sentences_of_test_in_oneshot_per_relation[relation] for relation in num_sentences_of_test_in_oneshot_per_relation if num_sentences_of_test_in_oneshot_per_relation[relation] != 0}
print(len(temp))
print(temp)
print(count_totals(temp))

False
0
{}
0

False
0
{}
0


In [39]:
# '\t'.join(line_temp[2:3]) --- x
print(any(num_sentences_of_oneshot_in_test_per_relation[relation] != 0 for relation in num_sentences_of_oneshot_in_test_per_relation))
temp = {relation: num_sentences_of_oneshot_in_test_per_relation[relation] for relation in num_sentences_of_oneshot_in_test_per_relation if num_sentences_of_oneshot_in_test_per_relation[relation] != 0}
print(len(temp))
print(temp)
print(count_totals(temp))
print()

print(any(num_sentences_of_test_in_oneshot_per_relation[relation] != 0 for relation in num_sentences_of_test_in_oneshot_per_relation))
temp = {relation: num_sentences_of_test_in_oneshot_per_relation[relation] for relation in num_sentences_of_test_in_oneshot_per_relation if num_sentences_of_test_in_oneshot_per_relation[relation] != 0}
print(len(temp))
print(temp)
print(count_totals(temp))

True
66
{'position played on team / speciality': 7135, 'parent company': 711, 'member of sports team': 5663, 'mouth of the watercourse': 4270, 'sport': 5158, 'drafted by': 7, 'taxon rank': 435, 'designer': 2642, 'place of birth': 13426, 'point in time': 325, 'voice type': 2099, 'architect': 5166, 'chromosome': 955, 'military rank': 32, 'illustrator': 564, 'father': 10489, 'time of discovery': 1264, 'place of death': 7551, 'instrument': 5993, 'manufacturer': 7015, 'creator': 5261, 'performer': 12793, 'production company': 2255, 'league': 692, 'participant of': 2220, 'country': 13834, 'member of political party': 2672, 'licensed to broadcast to': 19, 'noble family': 345, 'named after': 2720, 'nominated for': 54, 'child': 745, 'present in work': 2776, 'time of spacecraft launch': 2030, 'stock exchange': 2539, 'service entry': 1123, 'narrative location': 536, 'native language': 4468, 'connecting line': 178, 'screenwriter': 9268, 'series': 10209, 'site of astronomical discovery': 271, 'plac

# DISTINCT RELATION DESCRIPTIONS

In [10]:
relation_desc_train = defaultdict(lambda: [])
relation_desc_oneshot = defaultdict(lambda: [])
relation_desc_dev  = defaultdict(lambda: [])
relation_desc_eval = defaultdict(lambda: [])


relation_desc_counts_train = defaultdict(lambda: 0)
relation_desc_counts_oneshot = defaultdict(lambda: 0)
relation_desc_counts_dev  = defaultdict(lambda: 0)
relation_desc_counts_eval = defaultdict(lambda: 0)

distinct_relation_descs = defaultdict(lambda: 0)

#TODO: Tokenize sentences


with open(vlachos_train, "r", encoding='utf-8') as f:
    for line in f:
        line_temp = line.rstrip("\r\n").split("\t")
        distinct_relation_descs[line_temp[3]] += 1
        if (line_temp[3] not in relation_desc_train[line_temp[1]]):
            relation_desc_train[line_temp[1]].append(line_temp[3])
            relation_desc_counts_train[line_temp[1]] += 1

print(relation_desc_counts_train)

print("\n\n")


with open(vlachos_train_oneshot, "r", encoding='utf-8') as f:
    for line in f:
        line_temp = line.rstrip("\r\n").split("\t")
        distinct_relation_descs[line_temp[3]] += 1
        if (line_temp[3] not in relation_desc_oneshot[line_temp[1]]):
            relation_desc_oneshot[line_temp[1]].append(line_temp[3])
            relation_desc_counts_oneshot[line_temp[1]] += 1

print(relation_desc_counts_oneshot)

print("\n\n")


with open(vlachos_dev, "r", encoding='utf-8') as f:
    for line in f:
        line_temp = line.rstrip("\r\n").split("\t")
        distinct_relation_descs[line_temp[3]] += 1
        if (line_temp[3] not in relation_desc_dev[line_temp[1]]):
            relation_desc_dev[line_temp[1]].append(line_temp[3])
            relation_desc_counts_dev[line_temp[1]] += 1

print(relation_desc_counts_dev)

print("\n\n")


with open(vlachos_eval, "r", encoding='utf-8') as f:
    for line in f:
        line_temp = line.rstrip("\r\n").split("\t")
        distinct_relation_descs[line_temp[3]] += 1
        if (line_temp[3] not in relation_desc_eval[line_temp[1]]):
            relation_desc_eval[line_temp[1]].append(line_temp[3])
            relation_desc_counts_eval[line_temp[1]] += 1

print(relation_desc_counts_eval)

print("\n\n")

print(len(distinct_relation_descs))
#print(distinct_relation_descs)

defaultdict(<function <lambda> at 0x000002C1AE9F9598>, {'license': 1, 'military branch': 26, 'founder': 9, 'chairperson': 1, 'end time': 1, 'continent': 18, 'spouse': 1, 'constellation': 21, 'country of origin': 6, 'location of formation': 6, 'head of government': 4, 'based on': 2, 'vessel class': 4, 'IUCN conservation status': 8, 'located next to body of water': 3, 'airline hub': 9, 'brother': 2, 'date of death': 20, 'country of citizenship': 16, 'start time': 1, 'programming language': 3, 'record label': 4, 'film editor': 17, 'date of official opening': 14})



defaultdict(<function <lambda> at 0x000002C1AE9F9AE8>, {'position played on team / speciality': 28, 'parent company': 3, 'member of sports team': 15, 'mouth of the watercourse': 9, 'sport': 17, 'drafted by': 6, 'taxon rank': 2, 'designer': 9, 'place of birth': 14, 'point in time': 12, 'voice type': 8, 'architect': 29, 'chromosome': 10, 'military rank': 2, 'illustrator': 4, 'father': 14, 'time of discovery': 19, 'standards body

In [5]:
missing_train_x = 0
missing_train_x_n = 0
missing_train_y = 0
missing_train_b = 0
missing_oneshot_x = 0
missing_oneshot_x_n = 0
missing_oneshot_y = 0
missing_oneshot_b = 0
missing_dev_x = 0
missing_dev_x_n = 0
missing_dev_y = 0
missing_dev_b = 0
missing_eval_x = 0
missing_eval_x_n = 0
missing_eval_y = 0
missing_eval_b = 0

#TODO: Tokenize sentences


with open(vlachos_train, "r", encoding='utf-8') as f:
    for line in f:
        line_temp = line.rstrip("\r\n").split("\t")
        x = False
        y = False
        
        if ("SUBJECT_ENTITY" not in line_temp[2] or "OBJECT_ENTITY" not in line_temp[2]):
            x = True
            missing_train_x += 1
        if (("SUBJECT_ENTITY" not in line_temp[2] or "OBJECT_ENTITY" not in line_temp[2]) and line_temp[4] == "NEGATIVE"):
            missing_train_x_n += 1
        if ("SUBJECT_ENTITY" not in line_temp[3] or "OBJECT_ENTITY" not in line_temp[3]):
            y = True
            missing_train_y += 1
        if (x and y):
            missing_train_b += 1
            

print(missing_train_x, missing_train_x_n, missing_train_y, missing_train_b)

print("\n\n")




with open(vlachos_train_oneshot, "r", encoding='utf-8') as f:
    for line in f:
        line_temp = line.rstrip("\r\n").split("\t")
        x = False
        y = False
        
        if ("SUBJECT_ENTITY" not in line_temp[2] or "OBJECT_ENTITY" not in line_temp[2]):
            x = True
            missing_oneshot_x += 1
        if (("SUBJECT_ENTITY" not in line_temp[2] or "OBJECT_ENTITY" not in line_temp[2]) and line_temp[4] == "NEGATIVE"):
            missing_oneshot_x_n += 1
        if ("SUBJECT_ENTITY" not in line_temp[3] or "OBJECT_ENTITY" not in line_temp[3]):
            y = True
            missing_oneshot_y += 1
        if (x and y):
            missing_oneshot_b += 1
            

print(missing_oneshot_x, missing_oneshot_x_n, missing_oneshot_y, missing_oneshot_b)

print("\n\n")


with open(vlachos_dev, "r", encoding='utf-8') as f:
    for line in f:
        line_temp = line.rstrip("\r\n").split("\t")
        x = False
        y = False
        
        if ("SUBJECT_ENTITY" not in line_temp[2] or "OBJECT_ENTITY" not in line_temp[2]):
            x = True
            missing_dev_x += 1
        if (("SUBJECT_ENTITY" not in line_temp[2] or "OBJECT_ENTITY" not in line_temp[2]) and line_temp[4] == "NEGATIVE"):
            missing_dev_x_n += 1
        if ("SUBJECT_ENTITY" not in line_temp[3] or "OBJECT_ENTITY" not in line_temp[3]):
            y = True
            missing_dev_y += 1
        if (x and y):
            missing_dev_b += 1
            

print(missing_dev_x, missing_dev_x_n, missing_dev_y, missing_dev_b)

print("\n\n")


with open(vlachos_eval, "r", encoding='utf-8') as f:
    printed = False
    for line in f:
        line_temp = line.rstrip("\r\n").split("\t")
        x = False
        y = False
        
        if ("SUBJECT_ENTITY" not in line_temp[2] or "OBJECT_ENTITY" not in line_temp[2]):
            x = True
            missing_eval_x += 1
        if (("SUBJECT_ENTITY" not in line_temp[2] or "OBJECT_ENTITY" not in line_temp[2]) and line_temp[4] == "NEGATIVE"):
            missing_eval_x_n += 1
        if ("SUBJECT_ENTITY" not in line_temp[3] or "OBJECT_ENTITY" not in line_temp[3]):
            y = True
            missing_eval_y += 1
        if (x and y):
            missing_eval_b += 1
        
        if ((x or y) and not printed):
            printed = True
            print(line_temp)
            print(nltk.tokenize.word_tokenize(line_temp[2]))
        
            

print(missing_eval_x, missing_eval_x_n, missing_eval_y, missing_eval_b)

print("\n\n")

4574 4320 0 0



9342 9059 0 0



6907 6641 0 0



['832899', 'position played on team / speciality', 'SUBJECT_ENTITY was born William James Trew on 1 July 1886.', 'SUBJECT_ENTITY plays football in the capacity of OBJECT_ENTITY', 'NEGATIVE']
['SUBJECT_ENTITY', 'was', 'born', 'William', 'James', 'Trew', 'on', '1', 'July', '1886', '.']
9363 9063 0 0





# EXTRACT VLACHOS VALID SENTENCES / EXTRACT CORRESPONDING ENTITIES FROM LEVY

In [41]:
levy_data_finder1 = {}
levy_data_finder2 = {}
levy_data_finder3 = {}


for n, dataset_path in enumerate(levy_datasets_paths):
    print('\r' + '{:6.2f}%'.format(100*(n+1)/len(levy_datasets_paths)), end="", flush=True)
    with open(dataset_path, "r", encoding='utf-8') as f:
        for line in f:
            line_temp = line.rstrip("\r\n").split("\t")
            if (len(line_temp) >= 5):
                data = [line_temp[0]]

                s = line_temp[3]
                for i, entity in enumerate([re.compile(re.escape(line_temp[2]))] + [re.compile(re.escape(line_temp[i])) for i in range(4, len(line_temp))]):
                    s = re.sub(entity, 'ENTITY_' + str(i), s)

                s = nltk.tokenize.word_tokenize(s)
                data.append(s)

                for entity in [line_temp[2]] + [line_temp[i] for i in range(4, len(line_temp))]:
                    data.append(entity)

                if (s[0] not in levy_data_finder1):
                    levy_data_finder1[s[0]] = [data]
                else:
                    levy_data_finder1[s[0]].append(data)

                if (s[1] not in levy_data_finder2):
                    levy_data_finder2[s[1]] = [data]
                else:
                    levy_data_finder2[s[1]].append(data)

                if (s[2] not in levy_data_finder3):
                    levy_data_finder3[s[2]] = [data]
                else:
                    levy_data_finder3[s[2]].append(data)

100.00%

## FULL LEVY DATA

In [3]:
def file_len(fname):
    with open(fname, "r", encoding='utf-8') as f:
        for i, l in enumerate(f):
            pass
    return i + 1

In [4]:
print(file_len(path_datasets + 'Ours/sentences_only_test/normal/train.txt'))

6600


In [26]:
print(file_len(levy_full_datasets_paths[levy_full_datasets_names_id_map['positive']]))
print(file_len(levy_full_datasets_paths[levy_full_datasets_names_id_map['negative']]))

30633538
2249637


In [4]:
NUM_LINES_POSITIVE = 30633538
NUM_LINES_NEGATIVE = 2249637

# RELATION FILEPOINTERS

In [4]:
# Load relation names from file
relation_filepointers = {}

with open(path_relation_names, "r", encoding='utf-8') as f:
    for line in f:
        relation_filepointers[line.rstrip("\r\n")] = True

### Positive Examples (split by relation type)

##### There's a lot of repeated entries, for some reason, so we remove them a posteriori, after having renamed the files of the previous method

# Extend Levy's (already masked with entities) with Vlacho's relation descriptions

#### Start by getting and analysing Vlachos' relation descriptions

In [6]:
file_lengths = []
vlachos_relation_descriptions = {}

relation_desc_in_labels = {}

for i, dataset_path in enumerate(vlachos_datasets_paths):
    file_lengths.append(file_len(dataset_path))
    with open(dataset_path, "r", encoding='utf-8') as f:
        for l_num, line in enumerate(f):
            if (l_num % int(file_lengths[i]/1000) == 0 or l_num + 1 == file_lengths[i]): #TODO test efficiency of order in boolean tests with timeit
                print('\r' + str(i+1) + '/4' + ' ||| ' + str(l_num + 1) + '/' + str(file_lengths[i]), end="", flush=True)
            
            line_temp = line.rstrip("\r\n").split("\t")

            if (line_temp[4] != 'NEGATIVE'):
                if (line_temp[1] not in vlachos_relation_descriptions):
                    vlachos_relation_descriptions[line_temp[1]] = [line_temp[3]]
                else:
                    if (line_temp[3] not in vlachos_relation_descriptions[line_temp[1]]):
                        vlachos_relation_descriptions[line_temp[1]].append(line_temp[3])

                if (line_temp[3] not in relation_desc_in_labels):
                    relation_desc_in_labels[line_temp[3]] = [line_temp[1]]
                else:
                    if (line_temp[1] not in relation_desc_in_labels[line_temp[3]]):
                        relation_desc_in_labels[line_temp[3]].append(line_temp[1])

print()
print(file_lengths)
print('\n\n')

max_num_rel_descs = max(len(vlachos_relation_descriptions[relation]) for relation in vlachos_relation_descriptions) 
for i in range(1, max_num_rel_descs + 1):
    print('{:2d}'.format(i), '-', '{:2d}'.format(sum(len(vlachos_relation_descriptions[relation]) == i for relation in vlachos_relation_descriptions)), [relation for relation in vlachos_relation_descriptions if len(vlachos_relation_descriptions[relation]) == i])

4/4 ||| 580148/580148
[335898, 580204, 486024, 580148]



 1 - 14 ['license', 'chairperson', 'end time', 'spouse', 'start time', 'standards body', 'operating system', 'conferred by', 'collection', 'canonization status', 'characters', 'employer', 'industry', 'occupant']
 2 - 12 ['based on', 'brother', 'taxon rank', 'military rank', 'nominated for', 'child', 'narrative location', 'religious order', 'material used', 'residence', 'noble title', 'sister']
 3 - 12 ['located next to body of water', 'programming language', 'parent company', 'licensed to broadcast to', 'noble family', 'convicted of', 'occupation', 'replaced by', 'position held', 'located in the administrative territorial entity', 'found in taxon', 'architectural style']
 4 -  8 ['head of government', 'vessel class', 'record label', 'illustrator', 'named after', 'manner of death', 'editor', 'crosses']
 5 -  4 ['place of death', 'production company', 'parent taxon', 'award received']
 6 -  5 ['country of origin', 'location of for

In [7]:
# Some relation descriptions are present in more than 1 relation......

print(sum(len(relation_desc_in_labels[relation_desc]) for relation_desc in relation_desc_in_labels))
print(sum(len(vlachos_relation_descriptions[relation]) for relation in vlachos_relation_descriptions))
print(len(relation_desc_in_labels))
print()


for rel_desc in relation_desc_in_labels:
    if (len(relation_desc_in_labels[rel_desc]) > 1):
        print(rel_desc, end=': [')
        for r_num, relation in enumerate(relation_desc_in_labels[rel_desc]):
            print(('' if r_num == 0 else ', ') + relation + ": " + str(len(vlachos_relation_descriptions[relation])),
                  end=('' if r_num != len(relation_desc_in_labels[rel_desc]) - 1 else ']\n'))

1135
1135
1098

OBJECT_ENTITY created SUBJECT_ENTITY: [founder: 9, architect: 29, performer: 29, editor: 4, author: 27]
OBJECT_ENTITY is the name of the director of SUBJECT_ENTITY: [film editor: 17, director: 23]
OBJECT_ENTITY had the role of director in SUBJECT_ENTITY: [film editor: 17, director: 23]
OBJECT_ENTITY directed the film SUBJECT_ENTITY: [film editor: 17, director: 23]
OBJECT_ENTITY is the director of SUBJECT_ENTITY: [film editor: 17, screenwriter: 14, director: 23]
OBJECT_ENTITY was responsible for the direction of SUBJECT_ENTITY: [film editor: 17, screenwriter: 14, director: 23]
OBJECT_ENTITY directed SUBJECT_ENTITY: [film editor: 17, screenwriter: 14, director: 23]
The director of SUBJECT_ENTITY is OBJECT_ENTITY: [film editor: 17, director: 23]
OBJECT_ENTITY was the director of SUBJECT_ENTITY: [film editor: 17, screenwriter: 14, director: 23]
OBJECT_ENTITY was when SUBJECT_ENTITY was created: [date of official opening: 14, inception: 26]
OBJECT_ENTITY is the position of S

In [8]:
print(sum(len(vlachos_relation_descriptions[relation]) for relation in vlachos_relation_descriptions))
print(any(len(vlachos_relation_descriptions[relation]) == 0 for relation in vlachos_relation_descriptions))

1135
False


# Extract data, for easy hdf5 file saving by elmo

## Define some helpers

In [4]:
# Allows to get the number of lines in a file.
def file_len(fname):
    with open(fname, "r", encoding='utf-8') as f:
        for i, l in enumerate(f):
            pass
    return i + 1

In [4]:
# Load relation names from file.
relation_list = []

with open(path_relation_names, "r", encoding='utf-8') as f:
    for line in f:
        relation_list.append(line.strip())

#### Define a tokenizer

In [5]:
###########################################################################
## ELMO computes embeddings on pre-tokenized, space separated sentences! ##
###########################################################################
from allennlp.data.tokenizers.word_tokenizer import WordTokenizer
import pickle

tokenizer = WordTokenizer()

# Tokenize with a space between tokens.
def space_word_tokenize_string(s, tokenizer):
    tokenized_string = [token.text for token in tokenizer.tokenize(s)]
    sentence = []
    for k, token in enumerate(tokenized_string):
        sentence += [token] + ([' '] if k + 1 < len(tokenized_string) else [])
    return sentence

###########################################################################

### Levy's data
#### Here we produce all unique tuples of (sentence, relation, entities in relation).

In [5]:
# We define some imports specific to the data extraction.
import hashlib
from   ipywidgets import interact, fixed
from   IPython.display import clear_output
import time

# For printing purposes, we also get the number of lines present in Levy's positive examples.
NUM_LINES_POSITIVE = file_len(levy_full_datasets_paths[levy_full_datasets_names_id_map['positive']])

# We define a maximum sentence length.
max_sentence_length = 60

#### Here we get some statistics on those tuples, such as how many instances of the sentences appear in more than one relation and how many sentences appear more than once within the same relation, due to different entities. We also count the number of sentences assigned to each relation.

In [42]:
seen_xs = {}

num_instances = sum(file_len(path_datasets + 'Levy/by_relation/positive_' + str(max_sentence_length) + '/' + re.sub('/', '-', re.sub(' ', '_', relation)) + '.txt') for relation in relation_list)

repeated_xs = 0
repeated_xs_same_relation = 0

l_num = 1
for relation in relation_list:
    file_name_positive = path_datasets + 'Levy/by_relation/positive_' + str(max_sentence_length) + '/' + re.sub('/', '-', re.sub(' ', '_', relation)) + '.txt'
    with open(file_name_positive, 'r', encoding='utf-8') as f:
        for line in f:
            if (l_num % 500 == 0 or l_num + 1 == num_instances):
                print('\r' + '{:6.2f}%'.format(100*(l_num)/num_instances), end="", flush=True)
            l_num += 1

            x = line.strip().split('\t')[0]
            if (x in seen_xs):
                repeated_xs += 1
                if (relation in seen_xs[x]):
                    repeated_xs_same_relation += 1
                    seen_xs[x][relation] += 1
                else:
                    seen_xs[x][relation] = 1
            else:
                seen_xs[x] = {relation: 1}

print("\n")
print("Number of repetitions, across relation (the same sentence might be repeated more than once):", repeated_xs)
print("Number of sentences that appear more than once in the same relation:", repeated_xs_same_relation)

100.00%

Number of repetitions, across relation (the same sentence might be repeated more than once): 836239
Number of sentences that appear more than once in the same relation: 64


#### We get the data distribution per relation.

In [43]:
# Create a map from index to a tuple that indicates the relation and corresponding number of instances in that relation.
idx_to_relation_and_len = []


# Go over each relation and get the number of instances.
for r_num, relation in enumerate(relation_list):
    file_name_positive = path_datasets + 'Levy/by_relation/positive_' + str(max_sentence_length) + '/' + re.sub('/', '-', re.sub(' ', '_', relation)) + '.txt'
    idx_to_relation_and_len.append([relation, file_len(file_name_positive)])

# Sort the relations by number of instances, in increasing order.
idx_to_relation_and_len.sort(key=lambda lst: lst[1])
print(sum(lst[1] for lst in idx_to_relation_and_len))
print()
print(idx_to_relation_and_len)

# Create a map from relation to (sorted) index.
relation_to_idx = {relation: idx for idx, (relation, _) in enumerate(idx_to_relation_and_len)}

2314975

[['medical condition', 105], ['architectural style', 110], ['editor', 113], ['nominated for', 115], ['head of government', 126], ['date of official opening', 127], ['military rank', 127], ['time of discovery', 128], ['home venue', 139], ['replaced by', 143], ['instrumentation', 146], ['chairperson', 178], ['operating system', 182], ['product', 183], ['chromosome', 190], ['conferred by', 206], ['sister', 213], ['service entry', 218], ['religious order', 219], ['site of astronomical discovery', 219], ['manner of death', 222], ['convicted of', 228], ['license', 242], ['noble title', 251], ['standards body', 254], ['located next to body of water', 257], ['airline hub', 293], ['film editor', 308], ['illustrator', 326], ['time of spacecraft launch', 344], ['characters', 354], ['canonization status', 368], ['noble family', 392], ['location of formation', 404], ['collection', 414], ['IUCN conservation status', 420], ['drafted by', 440], ['industry', 468], ['based on', 502], ['located 

#### Now, we could allow for the same sentence, x, to be present in multiple relations (probably with different entities involved) or even for the same x to be present more than once for the same relation, for different entities. However, since at test time we do not really know how to mask (as that would mean knowing beforehand which entities are in the relation, which could happen, but is unlikely), we avoid having repeated sentences over the dataset. So, we assign those repeated sentences to the relation with the fewest instances.

In [44]:
# Make sure the "positive_'str(max_sentence_length)'_exclusive_and_unique_relation/" Directory exists.
path_dir_capped = path_datasets + 'Levy/by_relation/positive_' + str(max_sentence_length) + '/'
path_dir_capped_exclusive = path_datasets + 'Levy/by_relation/positive_' + str(max_sentence_length) + '_exclusive_and_unique_relation/'
directory = os.path.dirname(path_dir_capped_exclusive)
if (not os.path.exists(directory)):
    os.makedirs(directory)


relation_filepointers_capped_exclusive = {}

l_num = 1


for relation in relation_list:

    relation_path_capped = path_dir_capped + re.sub('/', '-', re.sub(' ', '_', relation)) + '.txt'
    relation_path_capped_exclusive = path_dir_capped_exclusive + re.sub('/', '-', re.sub(' ', '_', relation)) + '.txt'

    with open(relation_path_capped_exclusive, 'w', encoding='utf-8') as f_capped_exclusive:
        with open(relation_path_capped, 'r', encoding='utf-8') as f_capped:
            for line in f_capped:
                if (l_num % 500 == 0 or l_num + 1 == num_instances):
                    print('\r' + '{:6.2f}%'.format(100*(l_num)/num_instances), end="", flush=True)
                l_num += 1

                x = line.strip().split('\t')[0]

                try:
                    if (seen_xs[x] != 'DONE'):
                        if (len(seen_xs[x]) == 1):
                            f_capped_exclusive.write(line)
                            idx_to_relation_and_len[relation_to_idx[relation]][1] -= seen_xs[x][relation] - 1
                            seen_xs[x] = 'DONE'

                        else:
                            min_num_insts = np.inf
                            for repeated_inst_rel in seen_xs[x]:
                                if (idx_to_relation_and_len[relation_to_idx[repeated_inst_rel]][1] < min_num_insts):
                                    min_num_insts = idx_to_relation_and_len[relation_to_idx[repeated_inst_rel]][1]
                                    rel_with_fewest_num_insts = repeated_inst_rel

                            if (relation == rel_with_fewest_num_insts):
                                f_capped_exclusive.write(line)

                                for repeated_inst_rel in seen_xs[x]:
                                    if (relation == repeated_inst_rel):
                                        idx_to_relation_and_len[relation_to_idx[repeated_inst_rel]][1] -= seen_xs[x][repeated_inst_rel] - 1
                                    else:
                                        idx_to_relation_and_len[relation_to_idx[repeated_inst_rel]][1] -= seen_xs[x][repeated_inst_rel]
                                seen_xs[x] = 'DONE'
                            else:
                                seen_xs[x][relation] -= 1
                                idx_to_relation_and_len[relation_to_idx[relation]][1] -= 1

                except Exception as err:
                    # Here in case something needs to be printed.
                    raise err



print('\n')
idx_to_relation_and_len.sort(key=lambda lst: lst[1])
print("Total number of unique and exclusive sentences:", sum(lst[1] for lst in idx_to_relation_and_len))
print()
print(idx_to_relation_and_len)

100.00%

Total number of unique and exclusive sentences: 1472278

[['medical condition', 105], ['architectural style', 110], ['editor', 113], ['nominated for', 115], ['replaced by', 125], ['head of government', 126], ['date of official opening', 127], ['military rank', 127], ['time of discovery', 128], ['home venue', 139], ['instrumentation', 146], ['chairperson', 177], ['operating system', 182], ['product', 183], ['chromosome', 190], ['site of astronomical discovery', 195], ['conferred by', 206], ['sister', 212], ['service entry', 218], ['religious order', 219], ['manner of death', 222], ['convicted of', 228], ['license', 238], ['noble title', 251], ['standards body', 254], ['located next to body of water', 257], ['airline hub', 293], ['film editor', 308], ['illustrator', 325], ['time of spacecraft launch', 344], ['characters', 353], ['canonization status', 364], ['noble family', 392], ['location of formation', 404], ['collection', 414], ['IUCN conservation status', 420], ['drafted by

## TODO: Perform analysis of the data

### Now we actually create the datasets (with splits of train, validation and test), for the different settings and folds.

The settings in question are:

- Normal setting (N):
    - Draw instances according to the data distribution, but guaranteeing at least one instance per relation.


- Zero-Shot setting (ZS):
    - Classes are disjoint across the different splits. For each split we follow the data distribution, with the particularity that for the train split we guarantee at least 20 instances of each of the training classes (to deem them as 'seen' classes instead of 'unseen' and make the splits comparable to the Few-Shot scenario).


- Generalised Zero-Shot setting (GZS):
    - Similar to Zero-Shot, except train classes are also present in the validation and test splits (since in a real life scenario one would test against all existing classes). For the eval splits (val and test), 50% of the training instances are assigned to the seen classes (and then we follow the data distribution of those classes, guaranteeing at least 5 instances per class) and the other 50% are assigned to the unseen classes (and then we follow the data distribution of those classes, guaranteeing at least 5 instances per class).


- Few-Shot settings (with 1, 2, 5 and 10 shots) (FS-n): 
    - Similar to Zero-Shot, except now the train splits also have number-of-shots (1, 2, 5 or 10) instances of each class present in the eval splits. For the train splits we first guarantee the 'unseen' classes instances and, then, for the remaining missing number of instances we follow the data distribution of the 'seen' classes, with the particularity that we guarantee at least 20 instances of each of the training classes (to deem them as 'seen' classes instead of 'unseen').


- Generalised Few-Shot settings (with 1, 2, 5 and 10 shots) (GFS-n): 
    - Similar to Few-Shot, except train classes are also present in the validation and test splits (since in a real life scenario one would test against all existing classes). For the eval splits (val and test), 50% of the training instances are assigned to the 'seen' classes (and then we follow the data distribution of those classes, guaranteeing at least 5 instances per class) and the other 50% are assigned to the 'unseen' classes (and then we follow the data distribution of those classes, guaranteeing at least 5 instances per class).


---------------------
---------------------
---------------------

The datasets to be created are (where we define 'seen' classes as the 70 most populated classes and 'unseen' classes as the remainder of the classes):

- Final Evaluation ('final'):
    - This is the dataset where we test the final performance of our model. Some characteristics are as following:
        - This dataset type contains both 'seen' and 'unseen' classes (i.e. all classes. See the next dataset to understand why we make it explicit).
        - Each setting of this dataset contains 10 folds (each with train, validation and test splits) (so, 10 folds for N, 10 folds for ZS, 10 folds for GZS, ...).
        - There is no guarantee that the data is disjoint across distinct settings (see the next dataset to understand why we make it explicit).
        - For each fold we guarantee that the train, validation and test splits are disjoint from each other.
        - Across folds the train and validation splits are disjoint from one another.
        - We also guarantee that the validation and test splits, of all folds and settings, are completely disjoint from any data contained in the next dataset type ('hyperparameter'), which is used for Hyper-Parameter Tuning.
        - Finally, for each fold the splits are as follows, where BOTH 'seen' and 'unseen' classes are considered (each split is formed according to the specifications of the specific setting: read above):


            For settings other than the Normal setting:
              - 60 classes are sampled from the 'seen' set. These will be the train classes (disregarding specific setting configurations).
              - For both the validation and test splits we sample 20 classes from the 'unseen' set (20 for each split and assuring the classes in
                the validation split are disjoint from the classes in the test split). These will be the validation/test classes (disregarding
                specific setting configurations).
   
   
            - Train - 100.000 instances - Sampled according to the specifications of the specific setting: read above.
            - Val   - 5.000   instances - Sampled according to the specifications of the specific setting: read above.
            - test  - 25.000  instances - Sampled according to the specifications of the specific setting: read above.

<br>

- Hyper-Parameter Tuning ('hyperparameter'):
   - This is the dataset where we optimise the hyperparameters of the models. Some ablation experiments can also be performed here:
        - This dataset type contains only 'seen' classes. This is essential as we want to make sure that for the final evaluation the models will have never seen the 'unseen' classes, thus rendering it a true (Generalised) Any-Shot setting.
        - Each setting of this dataset contains 10 folds (each with train, validation and test splits) (so, 10 folds for N, 10 folds for ZS, 10 folds for GZS, ...).
        - We guarantee that the data is disjoint across distinct settings. This allows to select the hyperparameters that perform the best regardless of the setting. For our case this makes particular sense, as the whole idea revolves around a Natural Language Understanding/Inference paradigm. Therefore, if you perform well in one setting you should also perform well in a different one.
        - For each fold we guarantee that the train, validation and test splits are disjoint from each other.
        - Across folds ??? all splits are disjoint from one another? the train and validation splits are disjoint from one another?
        - Finally, for each fold the splits are as follows (each split is formed according to the specifications of the specific setting: read above):


            For settings other than the Normal setting:
              - 30 classes are sampled from the 'seen' set. These will be the train classes (disregarding specific setting configurations).
              - For both the validation and test splits we sample 15 classes from the 'seen' set. These will be the validation/test classes (disregarding
                specific setting configurations).
              - We guarantee that the classes sampled for any split are disjoint from the classes selected for any other split.
   
   
            - Train - 10.000 instances - Sampled according to the specifications of the specific setting: read above.
            - Val   - 500    instances - Sampled according to the specifications of the specific setting: read above.
            - test  - 2.500  instances - Sampled according to the specifications of the specific setting: read above.

<br>

- Debugging ('DEBUG'):
    - This is the dataset used to quickly debug models and training procedures:
        - This dataset type contains only 'seen' classes.
        - Regarding the (Generalised) Few-Shot settings, this dataset contains only 1-shot settings.
        - Each setting of this dataset contains 2 folds (each with train, validation and test splits) (so, 2 folds for N, 2 folds for ZS, 2 folds for GZS, ...).
        - We guarantee that the data is disjoint across distinct settings.
        - For each fold we guarantee that the train, validation and test splits are disjoint from each other.
        - Across folds all splits are disjoint from one another.
        - Finally, for each fold the splits are as follows (each split is formed according to the specifications of the specific setting: read above):


            For settings other than the Normal setting:
              - 5 classes are sampled from the 'seen' set. These will be the train classes (disregarding specific setting configurations).
              - For both the validation and test splits we sample 2 classes from the 'seen' set. These will be the validation/test classes (disregarding
                specific setting configurations).
              - We guarantee that the classes sampled for any split are disjoint from the classes selected for any other split.
   
   
            - Train - 20 instances - Sampled according to the specifications of the specific setting: read above.
            - Val   - 10 instances - Sampled according to the specifications of the specific setting: read above.
            - test  - 10 instances - Sampled according to the specifications of the specific setting: read above.


| Heading | Heading |
| ----| ----|
| text   | text |
| text | text |

In [7]:
# We start by defining a data sampler that guarantees a minimum number of instances.
def get_num_samples_and_guarantee_min_count(probs, num_samples, min_count, balanced=False):
    assert num_samples >= probs.shape[0]*min_count, "It's impossible to guarantee a minimum sample count for this total amount of samples."
    probs = probs.astype(float)

    if (balanced):
        counts = np.zeros(probs.shape) + num_samples // probs.shape[0]
        # In case 'num_samples % probs.shape[0] != 0' is True, that is, we cannot get the same number of samples on all classes, we
        # remove elements, at random, from the classes that have the most elements, making it as balanced as possible.
        while (counts.sum() != num_samples):
            counts[np.random.choice(numpy.where(counts == counts.max())[0])] -= 1

    else:
        probs /= probs.sum()

        counts = np.zeros(probs.shape)

        num_missing_below_threshold = (min_count - counts[counts < min_count]).sum()
        while (num_missing_below_threshold > 0 or counts.sum() != num_samples):
            if (num_samples - counts.sum() - num_missing_below_threshold == 0):
                counts[counts < min_count] = min_count
            else:
                counts += np.random.multinomial(num_samples - counts.sum() - num_missing_below_threshold, probs)
            num_missing_below_threshold = (min_count - counts[counts < min_count]).sum()

    # Assertions are redundant with while loop. Just a sanity check. REMOVE eventually?
    assert (counts < min_count).sum() == 0
    assert counts.sum() == num_samples
    return counts.astype(np.int64)

In [25]:
#################################################
# Define some dataset creation hyperparameters. #
#################################################

# Specify the maximum sentence length that is to be used.
max_sentence_length = 60

# Define the number of 'seen' classes. This will select the 'num_seen_classes' most populated classes as 'seen' classes.
num_seen_classes = 60
num_HT_seen_classes = 30

# Define the dataset type to be created (one of ['final_HT', 'DEBUG']).
dataset_composition_type = "final_HT"
dataset_types = ['HT', 'final'] if dataset_composition_type == "final_HT" else ['DEBUG']
print("Dataset Type to Compose:", dataset_composition_type)

# Define the settings to be created.
settings = ['N', 'ZS', 'GZS']
num_shots = [1, 2, 5, 10] if dataset_composition_type == 'final_HT' else [1] # This indicates the number of shots for the specific dataset type.
fs = lambda x: 'FS-' + str(x)
gfs = lambda x: 'GFS-' + str(x)
settings += [f(num_shot) for num_shot in num_shots for f in (fs, gfs)]
print("Settings to be evaluated:", settings)

# Define the number of folds, per dataset type.
num_folds = {'final': 10, 'HT': 5, 'DEBUG': 2}

# Define the size of each split, per dataset type.
splits     = {'train': 0, 'val': 1, 'test': 2}
split_size = {'final': {'train': 50000, 'val': 750, 'test': 1500},
              'HT'   : {'train': 10000, 'val': 375, 'test': 750},
              'DEBUG': {'train': 20,    'val': 10,  'test': 10}}


# Define the number of classes per split, per dataset type. This is only used in settings other than the normal one.
# These will be sampled from the entire set of corresponding classes (as per the setting).
num_classes = {'final': {'train': 60, 'val': 20, 'test': 20},
               'HT'   : {'train': 30, 'val': 10, 'test': 10},
               'DEBUG': {'train': 5,  'val': 2,  'test': 2}}



###########################
# Collect some statistics #
###########################

# Start by specifying the path to the data.
path_data_capped_length_exclusive = path_datasets + 'Levy/by_relation/positive_' + str(max_sentence_length) + '_exclusive_and_unique_relation/'

# Then get the data distribution, sort from fewest number of instances to highest number of instances.
idx_to_class_and_len = []

for c_num, _class in enumerate(relation_list):
    path_to_class_data = path_data_capped_length_exclusive + re.sub('/', '-', re.sub(' ', '_', _class)) + '.txt'

    idx_to_class_and_len.append([_class, file_len(path_to_class_data)])


idx_to_class_and_len.sort(key=lambda lst: lst[1], reverse=True)
class_to_idx = {_class: idx for idx, (_class, _) in enumerate(idx_to_class_and_len)}
classes_as_idx = list(range(len(class_to_idx)))
classes_num_insts = np.array([num_insts for (_, num_insts) in idx_to_class_and_len])

seen_classes = idx_to_class_and_len[:num_seen_classes]
seen_classes_to_seen_idx = {_class: idx for idx, (_class, _) in enumerate(seen_classes)}
seen_classes_as_idx = list(range(0, len(seen_classes_to_seen_idx)))

unseen_classes = idx_to_class_and_len[num_seen_classes:]
unseen_classes_to_seen_idx = {_class: idx for idx, (_class, _) in enumerate(unseen_classes)}
unseen_classes_as_idx = list(range(len(seen_classes_to_seen_idx), len(class_to_idx)))

# We compute the probability of the unseen classes, as sampling for FS-10 from a uniform class distribution makes it extremely hard to achieve a solution.
np_unseen_classes_lens = np.array([idx_to_class_and_len[idx][1] for idx in unseen_classes_as_idx], dtype=np.int64)
def softmax(x):
    return np.exp(x)/np.sum(np.exp(x))

unseen_classes_probs = np_unseen_classes_lens**(1/2)/np.sum(np_unseen_classes_lens**(1/2))#softmax(np_unseen_classes_lens/np.sum(np_unseen_classes_lens))

# HT_seen_classes = seen_classes[:num_HT_seen_classes]
# HT_seen_classes_to_seen_idx = {_class: idx for idx, (_class, _) in enumerate(HT_seen_classes)}
# HT_seen_classes_as_idx = list(range(0, len(HT_seen_classes_to_seen_idx)))

# HT_unseen_classes = seen_classes[num_HT_seen_classes:]
# HT_unseen_classes_to_seen_idx = {_class: idx for idx, (_class, _) in enumerate(HT_unseen_classes)}
# HT_unseen_classes_as_idx = list(range(len(HT_unseen_classes_to_seen_idx), len(seen_classes_to_seen_idx)))



print('Total number of instances:', sum(lst[1] for lst in idx_to_class_and_len))
print('\nDistribution of data instances per class:')
print()
print("'seen' classes (Total number of instances: " + str(sum(lst[1] for lst in seen_classes)) + '):\n', seen_classes)
print("\n\n'unseen' classes (Total number of instances: " + str(sum(lst[1] for lst in unseen_classes)) + '):\n', unseen_classes)
# print("\n\n\n")
# print("HT 'seen' classes (Total number of instances: " + str(sum(lst[1] for lst in HT_seen_classes)) + '):\n', HT_seen_classes)
# print("\n\nHT 'unseen' classes (Total number of instances: " + str(sum(lst[1] for lst in HT_unseen_classes)) + '):\n', HT_unseen_classes)

Dataset Type to Compose: final_HT
Settings to be evaluated: ['N', 'ZS', 'GZS', 'FS-1', 'GFS-1', 'FS-2', 'GFS-2', 'FS-5', 'GFS-5', 'FS-10', 'GFS-10']
Total number of instances: 1472278

Distribution of data instances per class:

'seen' classes (Total number of instances: 1449502):
 [['country', 223200], ['located in the administrative territorial entity', 218744], ['taxon rank', 151963], ['date of birth', 121276], ['occupation', 98026], ['date of death', 82778], ['country of citizenship', 54830], ['place of birth', 51100], ['parent taxon', 46210], ['performer', 40137], ['inception', 29735], ['sport', 27706], ['languages spoken or written', 25105], ['place of death', 23218], ['position played on team / speciality', 20781], ['author', 20104], ['publication date', 18215], ['director', 16990], ['member of sports team', 16395], ['headquarters location', 14965], ['cast member', 14424], ['participant of', 10287], ['sex or gender', 8946], ['member of political party', 7425], ['native language',

In [32]:
a = np.array([0, 0, 0])
classes = np.array(list(range(len(a))))

In [39]:
print(a)
print(softmax(a))
samples = np.random.choice(classes, size=2, replace=False, p=softmax(a))
print(samples)
a[samples] -= 1
print()
print(a)
print(softmax(a))

[-4 -4 -4]
[0.33333333 0.33333333 0.33333333]
[0 1]

[-5 -5 -4]
[0.21194156 0.21194156 0.57611688]


In [24]:
def softmax(x):
    return np.exp(x)/np.sum(np.exp(x))


print('\n\n\n\n')
N = 2
print(np.sum(np_unseen_classes_lens**(1/N)/np.sum(np_unseen_classes_lens**(1/N))))
print(np_unseen_classes_lens**(1/N)/np.sum(np_unseen_classes_lens**(1/N)))
print(softmax(np_unseen_classes_lens/np.sum(np_unseen_classes_lens)))






1.0000000000000002
[0.02706617 0.02609252 0.02422516 0.0236766  0.02355859 0.02333786
 0.02304602 0.02246868 0.02225503 0.02209343 0.0217117  0.02160139
 0.02132313 0.02130445 0.02115442 0.02111675 0.02088927 0.0206978
 0.02027019 0.02023087 0.01987347 0.01940723 0.01928375 0.01871799
 0.01828764 0.01815654 0.01793592 0.01766754 0.01702486 0.01676565
 0.01655054 0.01608698 0.0156606  0.01527449 0.01430538 0.01422164
 0.01413741 0.01376643 0.01347412 0.01329565 0.01320551 0.01317532
 0.01299274 0.01280757 0.01246092 0.01230013 0.01207142 0.0120384
 0.01187188 0.01078225 0.0105206  0.01009574 0.01005622 0.01005622
 0.01001655 0.00997672 0.00956934 0.00948576 0.009359   0.00914382]
[0.0170661  0.01701746 0.01692952 0.01690501 0.01689982 0.01689017
 0.01687757 0.01685314 0.01684426 0.0168376  0.01682209 0.01681766
 0.01680658 0.01680585 0.01679994 0.01679847 0.01678962 0.01678225
 0.01676605 0.01676458 0.01675133 0.01673442 0.01673002 0.01671019
 0.01669553 0.01669113 0.0166838  0.016

In [27]:
# We create the matrices that will allows us to count the number of instances.
dataset_composition_type = "final_HT"
dataset_types = ['HT', 'final'] if dataset_composition_type == "final_HT" else ['DEBUG']
num_instances_check = {dataset_type: {} for dataset_type in dataset_types}

classes_in_split = {dataset_type: {setting: {fold: {} for fold in range(num_folds[dataset_type])} for setting in settings} for dataset_type in dataset_types}

test_only_in_other_tests = True
HT_completely_disjoint = True

found_global_solution = False
num_tries_global_solution = 1


while (not found_global_solution):
    try:
        clear_output(True)
        print("TEST DISJOINTNESS TYPE:", "NOT in other folds' train/val splits." if test_only_in_other_tests else "in other folds' train/val splits.")
        print()

        for dataset_type in dataset_types:
            if (dataset_type == 'DEBUG'):
                seen_classes_debug = random.sample(seen_classes_as_idx, sum(num_classes[dataset_type][split] for split in splits))
                min_count_train = 1
                min_count_eval  = 1
            else:
                min_count_train = 20
                min_count_eval  = 5
                
            found_dataset_type_solution = False
            num_tries_dataset_type_solution = 1
            max_max_num_settings_verified = 0

            while (not found_dataset_type_solution):
                try:
                    if (dataset_type == 'HT'):
                        if (HT_completely_disjoint):
                            disjoint_HT = np.zeros(len(idx_to_class_and_len), dtype=np.int64)
                        else:
                            disjoint_train_val_HT = np.zeros(len(idx_to_class_and_len), dtype=np.int64)
                            if (test_only_in_other_tests):
                                test_HT_maximums = np.zeros(len(idx_to_class_and_len), dtype=np.int64)
                    maximum_number_of_settings_verified = 0
    
                    for setting_num, setting in enumerate(settings):
                        found_setting_solution = False
                        num_tries_setting_solution = 1
                        num_seen_classes_fails = 0
                        num_unseen_classes_fails = 0
                        max_verified_classes_so_far = 0

                        while (not found_setting_solution):
                            try:
                                if (dataset_type == 'HT' and HT_completely_disjoint):
                                    disjoint_HT_setting = np.zeros(len(idx_to_class_and_len), dtype=np.int64)
                                else:
                                    disjoint_train_setting = np.zeros(len(idx_to_class_and_len), dtype=np.int64)
                                    disjoint_val_setting   = np.zeros(len(idx_to_class_and_len), dtype=np.int64)
                                    if (test_only_in_other_tests):
                                        test_setting_maximums = np.zeros(len(idx_to_class_and_len), dtype=np.int64)

                                if (dataset_type != 'DEBUG'):
                                    num_instances_check[dataset_type][setting] = np.zeros((num_folds[dataset_type], 3, len(idx_to_class_and_len)), dtype=np.int64)

                                # Start by sampling the classes that will be present in this fold.
                                train_samples = np.zeros(num_seen_classes, dtype=np.int64)
                                val_samples   = np.zeros(len(classes_as_idx) - num_seen_classes)
                                test_samples  = np.zeros(len(classes_as_idx) - num_seen_classes)
                                for fold in range(num_folds[dataset_type]):
                                    if (setting == 'N'):
                                        for split in splits:
                                            if (dataset_type == 'DEBUG'):
                                                classes_in_split[dataset_type][setting][fold][split] = list(seen_classes_debug)
                                            elif (dataset_type == 'HT'):
                                                classes_in_split[dataset_type][setting][fold][split] = list(seen_classes_as_idx)
                                            else:
                                                classes_in_split[dataset_type][setting][fold][split] = list(classes_as_idx)
                                    else:
                                        if (dataset_type == 'DEBUG'):
                                            sampled_classes = random.sample(seen_classes_debug, sum(num_classes[dataset_type][split] for split in splits))

                                            train_classes = sampled_classes[:num_classes[dataset_type]['train']]
                                            val_classes   = sampled_classes[num_classes[dataset_type]['train']:num_classes[dataset_type]['train']+num_classes[dataset_type]['val']]
                                            test_classes  = sampled_classes[-num_classes[dataset_type]['test']:]
                                            
                                        elif (dataset_type == 'HT'):
                                            sampled_classes = random.sample(seen_classes_as_idx, sum(num_classes[dataset_type][split] for split in splits))

                                            train_classes = sampled_classes[:num_classes[dataset_type]['train']]
                                            val_classes   = sampled_classes[num_classes[dataset_type]['train']:num_classes[dataset_type]['train']+num_classes[dataset_type]['val']]
                                            test_classes  = sampled_classes[-num_classes[dataset_type]['test']:]
                                        else:
#                                             train_classes = random.sample(seen_classes_as_idx, num_classes[dataset_type]['train'])
                                            samples = np.random.choice(np.array(seen_classes_as_idx), size=num_classes[dataset_type]['train'],
                                                                       replace=False, p=softmax(train_samples))
                                            train_samples[samples] -= 1
                                            train_classes = list(samples)
#                                             if (setting == 'FS-10'):
#                                                 sampled_unseen_classes = list(np.random.choice(np.array(unseen_classes_as_idx),
#                                                                                                size=num_classes[dataset_type]['val'] + num_classes[dataset_type]['test'],
#                                                                                                replace=False, p=unseen_classes_probs))
#                                             else:
#                                                 sampled_unseen_classes = random.sample(unseen_classes_as_idx, num_classes[dataset_type]['val'] + num_classes[dataset_type]['test'])
                                            if (setting == 'FS-10'):
                                                val_probs = softmax(val_samples) + unseen_classes_probs
                                                val_probs = val_probs/np.sum(val_probs)
                                            else:
                                                val_probs = softmax(val_samples)
                                            samples = np.random.choice(np.array(unseen_classes_as_idx)-num_seen_classes, size=num_classes[dataset_type]['val'],
                                                                       replace=False, p=val_probs)
                                            val_samples[samples] -= 1
                                            val_classes = list(samples + num_seen_classes)

                                            if (setting == 'FS-10'):
                                                test_probs = np.array([test_sample for i, test_sample in enumerate(softmax(test_samples) + unseen_classes_probs) if i not in samples])
                                                test_probs = test_probs/np.sum(test_probs)
                                            else:
                                                test_probs = softmax(np.array([test_sample for i, test_sample in enumerate(test_samples) if i not in samples]))
                                            samples = np.random.choice(np.array([idx for idx in unseen_classes_as_idx if idx not in (samples + num_seen_classes)])-num_seen_classes,
                                                                       size=num_classes[dataset_type]['test'],
                                                                       replace=False,
                                                                       p=test_probs)
                                            test_samples[samples] -= 1
                                            test_classes = list(samples + num_seen_classes)

                                        classes_in_split[dataset_type][setting][fold]['train'] = train_classes
                                        classes_in_split[dataset_type][setting][fold]['val']   = val_classes
                                        classes_in_split[dataset_type][setting][fold]['test']  = test_classes

#                                 # Check that all classes are present in this setting.
#                                 classes_present_in_setting = np.sum(np.sum(num_instances_check[dataset_type][setting], axis=1), axis=0) != 0
#                                 err_msg = "\rASSERT ALL CLASSES IN SETTING: " + dataset_type + ", " + setting + " | Num classes in setting:  "
#                                 err_msg += str(np.sum(classes_present_in_setting)) + "."
#                                 assert np.sum(classes_present_in_setting) == (num_seen_classes if dataset_type == 'HT' else 120), err_msg


                                # Check that the class distribution for this fold is approximately uniform.
                                if (dataset_type == 'final' and setting != 'N'):
                                    classes_train_uniform_check = np.zeros(num_seen_classes, dtype=np.int8)
                                    classes_val_uniform_check   = np.zeros(len(idx_to_class_and_len), dtype=np.int8)
                                    classes_test_uniform_check  = np.zeros(len(idx_to_class_and_len), dtype=np.int8)

                                    for fold in range(num_folds[dataset_type]):
                                        classes_train_uniform_check[classes_in_split[dataset_type][setting][fold]['train']] += 1
                                        classes_val_uniform_check[classes_in_split[dataset_type][setting][fold]['val']] += 1
                                        classes_test_uniform_check[classes_in_split[dataset_type][setting][fold]['test']] += 1

                                    train_diff = np.max(classes_train_uniform_check) - np.min(classes_train_uniform_check)
                                    err_msg = setting + " does not have an approximate uniform class distribution. Train Diff: " + str(train_diff)
                                    err_msg += " | (max: " + str(max_verified_classes_so_far) + ")"
                                    assert train_diff <= 3, err_msg
                                    err_msg = setting + " does not have at least 2 times each class for train: "
                                    err_msg += str(num_seen_classes - np.sum(classes_train_uniform_check[:num_seen_classes] >= 2))
                                    err_msg += " | (max: " + str(max_verified_classes_so_far) + ")"
                                    assert (classes_train_uniform_check[:num_seen_classes] >= 2).all(), err_msg
        
                                    val_diff = np.max(classes_val_uniform_check[num_seen_classes:]) - np.min(classes_val_uniform_check[num_seen_classes:])
                                    err_msg = setting + " does not have an approximate uniform class distribution. Val Diff: " + str(val_diff)
                                    err_msg += " | (max: " + str(max_verified_classes_so_far) + ")"
                                    assert val_diff <= 3, err_msg
                                    err_msg = setting + " does not have at least 2 times each class for val: "
                                    err_msg += str((len(idx_to_class_and_len) - num_seen_classes) - np.sum(classes_val_uniform_check[num_seen_classes:] >= 2))
                                    err_msg += " | (max: " + str(max_verified_classes_so_far) + ")"
                                    assert (classes_val_uniform_check[num_seen_classes:] >= 2).all(), err_msg
        
                                    test_diff = np.max(classes_test_uniform_check[num_seen_classes:]) - np.min(classes_test_uniform_check[num_seen_classes:])
                                    err_msg = setting + " does not have an approximate uniform class distribution. Test Diff: " + str(test_diff)
                                    err_msg += " | (max: " + str(max_verified_classes_so_far) + ")"
                                    assert test_diff <= 3, err_msg
                                    err_msg = setting + " does not have at least 2 times each class for test: "
                                    err_msg += str((len(idx_to_class_and_len) - num_seen_classes) - np.sum(classes_test_uniform_check[num_seen_classes:] >= 2))
                                    err_msg += " | (max: " + str(max_verified_classes_so_far) + ")"
                                    assert (classes_test_uniform_check[num_seen_classes:] >= 2).all(), err_msg
                                

                                # Now we sample the number of instances for this fold.
                                for fold in range(num_folds[dataset_type]):
                                    if (setting == 'N' or setting == 'ZS'):
                                        for split in splits:
                                            if (split != 'train'):
                                                min_count = min_count_eval
                                            else:
                                                min_count = 1 if setting == 'N' else min_count_train
                                            samples = get_num_samples_and_guarantee_min_count(classes_num_insts[classes_in_split[dataset_type][setting][fold][split]],
                                                                                              split_size[dataset_type][split], min_count=min_count)
                                            num_instances_check[dataset_type][setting][fold, splits[split], classes_in_split[dataset_type][setting][fold][split]] = samples

                                    elif (setting == 'GZS'):
                                        for split in splits:
                                            min_count = min_count_train if split == 'train' else min_count_eval
                                            if (split == 'train'):
                                                samples = get_num_samples_and_guarantee_min_count(classes_num_insts[classes_in_split[dataset_type][setting][fold][split]],
                                                                                                  split_size[dataset_type][split], min_count=min_count)
                                                num_instances_check[dataset_type][setting][fold, splits[split], classes_in_split[dataset_type][setting][fold][split]] = samples
                                            else:
                                                t_samples = get_num_samples_and_guarantee_min_count(classes_num_insts[classes_in_split[dataset_type][setting][fold]['train']],
                                                                                                    int(0.5*split_size[dataset_type][split]), min_count=min_count_eval)
                                                e_samples = get_num_samples_and_guarantee_min_count(classes_num_insts[classes_in_split[dataset_type][setting][fold][split]],
                                                                                                    split_size[dataset_type][split] - int(0.5*split_size[dataset_type][split]),
                                                                                                    min_count=min_count_eval)
                                                num_instances_check[dataset_type][setting][fold, splits[split], classes_in_split[dataset_type][setting][fold]['train']] = t_samples
                                                num_instances_check[dataset_type][setting][fold, splits[split], classes_in_split[dataset_type][setting][fold][split]]   = e_samples

                                    else: # (Generalised) Few-Shot Setting.
                                        num_shots = int(setting.split('-')[1])
                                        t_samples_from_val = get_num_samples_and_guarantee_min_count(classes_num_insts[classes_in_split[dataset_type][setting][fold]['val']],
                                                                                                     num_shots*len(classes_in_split[dataset_type][setting][fold]['val']),
                                                                                                     min_count=num_shots)
                                        t_samples_from_tst = get_num_samples_and_guarantee_min_count(classes_num_insts[classes_in_split[dataset_type][setting][fold]['test']],
                                                                                                     num_shots*len(classes_in_split[dataset_type][setting][fold]['test']),
                                                                                                     min_count=num_shots)
                                        eval_num_samples = np.sum(t_samples_from_val) + np.sum(t_samples_from_tst)
                                        t_samples_from_trn = get_num_samples_and_guarantee_min_count(classes_num_insts[classes_in_split[dataset_type][setting][fold]['train']],
                                                                                                     split_size[dataset_type]['train'] - eval_num_samples,
                                                                                                     min_count=min_count_train)
                                        num_instances_check[dataset_type][setting][fold, splits['train'], classes_in_split[dataset_type][setting][fold]['train']] = t_samples_from_trn
                                        num_instances_check[dataset_type][setting][fold, splits['train'], classes_in_split[dataset_type][setting][fold]['val']]   = t_samples_from_val
                                        num_instances_check[dataset_type][setting][fold, splits['train'], classes_in_split[dataset_type][setting][fold]['test']]  = t_samples_from_tst

                                        for split in splits:
                                            if (split == 'train'):
                                                continue
                                            else:
                                                if (setting[0] == 'F'): # Non-Generalised Setting
                                                    samples = get_num_samples_and_guarantee_min_count(classes_num_insts[classes_in_split[dataset_type][setting][fold][split]],
                                                                                                      split_size[dataset_type][split], min_count=min_count_eval)
                                                    num_instances_check[dataset_type][setting][fold, splits[split], classes_in_split[dataset_type][setting][fold][split]] = samples

                                                else: # Generalised Setting
                                                    t_samples = get_num_samples_and_guarantee_min_count(classes_num_insts[classes_in_split[dataset_type][setting][fold]['train']],
                                                                                                        int(0.5*split_size[dataset_type][split]), min_count=min_count_eval)
                                                    e_samples = get_num_samples_and_guarantee_min_count(classes_num_insts[classes_in_split[dataset_type][setting][fold][split]],
                                                                                                        split_size[dataset_type][split] - int(0.5*split_size[dataset_type][split]),
                                                                                                        min_count=min_count_eval)
                                                    num_instances_check[dataset_type][setting][fold, splits[split], classes_in_split[dataset_type][setting][fold]['train']] = t_samples
                                                    num_instances_check[dataset_type][setting][fold, splits[split], classes_in_split[dataset_type][setting][fold][split]]   = e_samples


                                    # Verify that it is possible to build this fold.
                                    if (dataset_type == 'HT'):
                                        fold_num_insts = np.sum(num_instances_check[dataset_type][setting][fold], axis=0)
                                    elif (dataset_type == 'final'):
                                        if (HT_completely_disjoint):
                                            fold_train_num_insts = np.maximum(disjoint_HT, num_instances_check[dataset_type][setting][fold][splits['train']])
                                        else:
                                            fold_train_num_insts = np.maximum(disjoint_train_val_HT + test_HT_maximums, num_instances_check[dataset_type][setting][fold][splits['train']])
                                        fold_num_insts = fold_train_num_insts + np.sum(num_instances_check[dataset_type][setting][fold][1:], axis=0)

                                    verify_fold_less_equal = np.less_equal(fold_num_insts, classes_num_insts)
                                    err_msg = dataset_type + ", " + setting + ", " + str(fold) + ": " + str(np.sum(verify_fold_less_equal))
                                    if (dataset_type == 'final'):
                                        if (not all(verify_fold_less_equal[:len(seen_classes_as_idx)])):
                                            num_seen_classes_fails += 1
                                        if (not all(verify_fold_less_equal[len(seen_classes_as_idx):])):
                                            num_unseen_classes_fails += 1
                                    assert verify_fold_less_equal.all(), err_msg

                                    # Increment counts to test for setting disjointness.
                                    if (dataset_type == 'HT' and HT_completely_disjoint):
                                        disjoint_HT_setting += fold_num_insts
                                    else:
                                        disjoint_train_setting += num_instances_check[dataset_type][setting][fold][splits['train']]
                                        disjoint_val_setting   += num_instances_check[dataset_type][setting][fold][splits['val']]
        #                                     if (dataset_type == 'HT'):
        #                                         disjoint_test_setting += num_instances_check[dataset_type][setting][fold][splits['test']]
        #                                     if (dataset_type == 'final'):
                                        if (test_only_in_other_tests):
                                            test_setting_maximums = np.maximum(test_setting_maximums, num_instances_check[dataset_type][setting][fold][splits['test']])

                                # Now verify that this entire setting is possible by itself.
                                if (test_only_in_other_tests):
                                    if (dataset_type == 'HT'):
                                        if (HT_completely_disjoint):
                                            total_setting_num_instances = disjoint_HT_setting 
                                        else:
                                            total_setting_num_instances = disjoint_train_setting + disjoint_val_setting + test_setting_maximums
                                    elif (dataset_type == 'final'):
                                        # This accounts for the train set being allowed to reuse instances from the 'HT' dataset.
                                        if (HT_completely_disjoint):
                                            setting_train_num_insts = np.maximum(disjoint_HT, disjoint_train_setting)
                                        else:
                                            setting_train_num_insts = np.maximum(disjoint_train_val_HT + test_HT_maximums, disjoint_train_setting)
                                        total_setting_num_instances = setting_train_num_insts + disjoint_val_setting + test_setting_maximums
                                verify_setting_less_equal = np.less_equal(total_setting_num_instances, classes_num_insts)
                                
                                if (dataset_type == 'final'):
                                    if (not all(verify_setting_less_equal[:len(seen_classes_as_idx)])):
                                        num_seen_classes_fails += 1
                                    if (not all(verify_setting_less_equal[len(seen_classes_as_idx):])):
                                        num_unseen_classes_fails += 1
                                
                                num_verified_classes = np.sum(verify_setting_less_equal)
                                if (max_verified_classes_so_far < num_verified_classes):
                                    max_verified_classes_so_far = num_verified_classes


                                err_msg = "(" + dataset_type + ", " + setting + ") verified number of relations: " + str(np.sum(verify_setting_less_equal))
                                err_msg += " (max: " + str(max_verified_classes_so_far) + ")"
                                assert verify_setting_less_equal.all(), err_msg

                                found_setting_solution = True

                            except AssertionError as err:
                                if (not found_setting_solution):
                                    if (dataset_type == 'HT'):
                                        if (num_tries_setting_solution == 50):
                                            raise AssertionError(err)
                                    elif (dataset_type == 'final'):
                                        if (num_tries_setting_solution >= 2000 and num_seen_classes_fails/num_tries_setting_solution > 0.2):
                                            raise AssertionError(err)
                                    if (dataset_type == 'final' and (num_tries_setting_solution - 1) % 250 == 0):
                                        print('\r' + str(err) + " | Number of attempts: " + str(num_tries_setting_solution) +
                                              " ('seen' fails: " + str(num_seen_classes_fails) +
                                              ' ({:.2f}%)'.format(100*num_seen_classes_fails/num_tries_setting_solution) + ")" +
                                              " ('unseen' fails: " + str(num_unseen_classes_fails) +
                                              ' ({:.2f}%)'.format(100*num_unseen_classes_fails/num_tries_setting_solution) + ")",
                                              end="", flush=True)
                                    num_tries_setting_solution += 1

                        # Increment counts to test for 'HT' disjointness and check for disjointness.
                        if (dataset_type == 'HT'):
                            if (HT_completely_disjoint):
                                disjoint_HT += disjoint_HT_setting
                                verify_disjoint_HT = disjoint_HT
                            else:
                                disjoint_train_val_HT += disjoint_train_setting + disjoint_val_setting
                                if (test_only_in_other_tests):
                                    test_HT_maximums = np.maximum(test_HT_maximums, test_setting_maximums)
                                verify_disjoint_HT = disjoint_train_val_HT + test_HT_maximums

                            # Here we check if it is still possible (with this new setting) to create the dataset_type 'HT'.
                            verify_HT_settings_less_equal = np.less_equal(verify_disjoint_HT, classes_num_insts)
                            err_msg = dataset_type + " (verified number of settings before failure): " + str(setting_num) + "/" + str(len(settings)) + "."
                            err_msg += " | " + str(maximum_number_of_settings_verified)
                            assert verify_HT_settings_less_equal.all(), err_msg

                        maximum_number_of_settings_verified += 1

                        if (max_max_num_settings_verified < maximum_number_of_settings_verified):
                            max_max_num_settings_verified = maximum_number_of_settings_verified

                        if (dataset_type == 'HT'):
                            print("\rDataset: " + dataset_type + " | Maximum Verified Settings: " + str(settings[:max_max_num_settings_verified]) +
                                  " (" + str(maximum_number_of_settings_verified) + ")", end="", flush=True)
                        if (dataset_type == 'final'):
                            print("\rFound solution for (" + dataset_type + ", " + setting + ") after " + str(num_tries_setting_solution) + " iterations.")
                            
                        if (maximum_number_of_settings_verified == len(settings)):
                            found_dataset_type_solution = True

                except AssertionError as err:
                    if (not found_dataset_type_solution):
                        if (dataset_type == 'final'):
                            raise RuntimeError(err)
                        num_tries_dataset_type_solution += 1


            print("\nFound solution for (" + dataset_type + ") after " + str(num_tries_dataset_type_solution) + " iterations.")
            if (dataset_type == 'final'):
                found_global_solution = True

    except RuntimeError as err:
        if (not found_global_solution):
            clear_output()
            num_tries_global_solution += 1


                   
np.set_printoptions(formatter={'all':lambda x: '{:5d}'.format(x)})
def specific_dataset_print(dataset_type, setting, fold, split, mask_train):
    # The interact module automatically retrieves the dictionary values, for the corresponding keys.

    print("\n"*5)
    if (setting != 'N'):
        train_mask = (num_instances_check[dataset_type][setting][fold][0] == 0).astype(np.int8)
        if (setting[:3] == 'GFS' or setting[:3] == 'FS-'):
            train_mask += (num_instances_check[dataset_type][setting][fold][0] == int(setting.split('-')[1])).astype(np.int8)
    else:
        train_mask = np.ones_like(num_instances_check[dataset_type][setting][fold][0], dtype=np.int8)

    if (setting[:3] == 'GFS' or setting[:3] == 'FS-'):
        anti_train_mask = (~train_mask.astype(np.bool)).astype(np.int8)
    else:
        anti_train_mask = np.ones_like(num_instances_check[dataset_type][setting][fold][0], dtype=np.int8)
        

    masked = np.multiply(train_mask, num_instances_check[dataset_type][setting][fold][split])
    anti_masked = np.multiply(anti_train_mask, num_instances_check[dataset_type][setting][fold][split])

    print('Fold num insts:', sum(np.sum(num_instances_check[dataset_type][setting][fold][split_num]) for split_num in [0, 1, 2]))
    print('Split num insts:', np.sum(num_instances_check[dataset_type][setting][fold][split]))
    print("'Exclusive' split num insts:", np.sum(masked if split != 0 else anti_masked))
#     print("'seen' classes distribution train:", np.sum(num_instances_check[dataset_type][setting][fold_num][0] != 0 for fold_num in range(num_folds[dataset_type])))
#     print("'unseen' classes distribution val:", np.sum(num_instances_check[dataset_type][setting][fold_num][1] != 0 for fold_num in range(num_folds[dataset_type])))
#     print("'unseen' classes distribution test:", np.sum(num_instances_check[dataset_type][setting][fold_num][2] != 0 for fold_num in range(num_folds[dataset_type])))
    print()
    print(masked if mask_train else num_instances_check[dataset_type][setting][fold][split])

def select_dataset_type_print(dataset_type):
    global settings
    global num_folds
    global splits
    interact(specific_dataset_print, dataset_type=fixed(dataset_type), setting=settings,
             fold=list(range(num_folds[dataset_type])), split=splits, mask_train=False)
    
interact(select_dataset_type_print, dataset_type=dataset_types);

TEST DISJOINTNESS TYPE: NOT in other folds' train/val splits.

Dataset: HT | Maximum Verified Settings: ['N', 'ZS', 'GZS', 'FS-1', 'GFS-1', 'FS-2', 'GFS-2', 'FS-5', 'GFS-5', 'FS-10', 'GFS-10'] (11)
Found solution for (HT) after 2 iterations.
Found solution for (final, N) after 1 iterations.
Found solution for (final, ZS) after 1 iterations.
Found solution for (final, GZS) after 1 iterations.
Found solution for (final, FS-1) after 1 iterations.
Found solution for (final, GFS-1) after 1 iterations.
Found solution for (final, FS-2) after 1 iterations.
Found solution for (final, GFS-2) after 1 iterations.
Found solution for (final, FS-5) after 5 iterations.
Found solution for (final, GFS-5) after 1 iterations.
FS-10 does not have an approximate uniform class distribution. Val Diff: 4 | (max: 118) | Number of attempts: 229751 ('seen' fails: 0 (0.00%)) ('unseen' fails: 4719 (2.05%))

KeyboardInterrupt: 

In [56]:
with open('temp_files.ckp', 'wb') as f:
    pickle.dump(num_instances_check, f)
    pickle.dump(classes_in_split, f)

In [54]:
np.set_printoptions(formatter={'all':lambda x: '{:5d}'.format(x)})
def specific_dataset_print(dataset_type, setting, fold, split, mask_train):
    # The interact module automatically retrieves the dictionary values, for the corresponding keys.

    print("\n"*5)
    if (setting != 'N'):
        train_mask = (num_instances_check[dataset_type][setting][fold][0] == 0).astype(np.int8)
        if (setting[:3] == 'GFS' or setting[:3] == 'FS-'):
            train_mask += (num_instances_check[dataset_type][setting][fold][0] == int(setting.split('-')[1])).astype(np.int8)
    else:
        train_mask = np.ones_like(num_instances_check[dataset_type][setting][fold][0], dtype=np.int8)

    if (setting[:3] == 'GFS' or setting[:3] == 'FS-'):
        anti_train_mask = (~train_mask.astype(np.bool)).astype(np.int8)
    else:
        anti_train_mask = np.ones_like(num_instances_check[dataset_type][setting][fold][0], dtype=np.int8)
        

    masked = np.multiply(train_mask, num_instances_check[dataset_type][setting][fold][split])
    anti_masked = np.multiply(anti_train_mask, num_instances_check[dataset_type][setting][fold][split])

    print('Fold num insts:', sum(np.sum(num_instances_check[dataset_type][setting][fold][split_num]) for split_num in [0, 1, 2]))
    print('Split num insts:', np.sum(num_instances_check[dataset_type][setting][fold][split]))
    print("'Exclusive' split num insts:", np.sum(masked if split != 0 else anti_masked))
    print("'seen' classes distribution train:")
    dist_seen = np.sum(num_instances_check[dataset_type][setting][fold_num][0][:num_seen_classes] != 0 for fold_num in range(num_folds[dataset_type]))
    print("Diff max-min:", np.max(dist_seen) - np.min(dist_seen))
    print(dist_seen)
    print("'unseen' classes distribution val:")
    dist_unseen_val = np.sum(num_instances_check[dataset_type][setting][fold_num][1][num_seen_classes:] != 0 for fold_num in range(num_folds[dataset_type]))
    print("Diff max-min:", np.max(dist_unseen_val) - np.min(dist_unseen_val))
    print(dist_unseen_val)
    print("'unseen' classes distribution test:")
    dist_unseen_test = np.sum(num_instances_check[dataset_type][setting][fold_num][2][num_seen_classes:] != 0 for fold_num in range(num_folds[dataset_type]))
    print("Diff max-min:", np.max(dist_unseen_test) - np.min(dist_unseen_test))
    print(dist_unseen_test)
    print()
    print(masked if mask_train else num_instances_check[dataset_type][setting][fold][split])

def select_dataset_type_print(dataset_type):
    global settings
    global num_folds
    global splits
    interact(specific_dataset_print, dataset_type=fixed(dataset_type), setting=settings,
             fold=list(range(num_folds[dataset_type])), split=splits, mask_train=False)

interact(select_dataset_type_print, dataset_type=dataset_types);

interactive(children=(Dropdown(description='dataset_type', options=('HT', 'final'), value='HT'), Output()), _d…

In [28]:
# We create the matrices that will allows us to count the number of instances.
dataset_composition_type = "final_HT"
dataset_types = ['HT', 'final'] if dataset_composition_type == "final_HT" else ['DEBUG']
num_instances_check = {dataset_type: {} for dataset_type in dataset_types}

classes_in_split = {dataset_type: {setting: {fold: {} for fold in range(num_folds[dataset_type])} for setting in settings} for dataset_type in dataset_types}

test_only_in_other_tests = True

found_global_solution = False
num_tries_global_solution = 1


while (not found_global_solution):
    try:
        clear_output(True)
        print("TEST DISJOINTNESS TYPE:", "in other folds' train/val splits." if test_only_in_other_tests else "NOT in other folds' train/val splits.")
        print()

        for dataset_type in dataset_types:
            if (dataset_type == 'DEBUG'):
                seen_classes_debug = random.sample(seen_classes_as_idx, sum(num_classes[dataset_type][split] for split in splits))
                min_count_train = 1
                min_count_eval  = 1
            else:
                min_count_train = 20
                min_count_eval  = 5
                
            found_dataset_type_solution = False
            num_tries_dataset_type_solution = 1

            while (not found_dataset_type_solution):
                try:
                    if (dataset_type == 'HT'):
                        disjoint_train_val_HT = np.zeros(len(idx_to_class_and_len), dtype=np.int64)
                        if (test_only_in_other_tests):
                            test_HT_maximums = np.zeros(len(idx_to_class_and_len), dtype=np.int64)
                        maximum_number_of_settings_verified = 0
    
                    for setting_num, setting in enumerate(settings):
                        found_setting_solution = False
                        num_tries_setting_solution = 1
                        max_verified_classes_so_far = 0

                        while (not found_setting_solution):
                            try:
                                disjoint_train_setting = np.zeros(len(idx_to_class_and_len), dtype=np.int64)
                                disjoint_val_setting   = np.zeros(len(idx_to_class_and_len), dtype=np.int64)
                                if (test_only_in_other_tests):
                                    test_setting_maximums = np.zeros(len(idx_to_class_and_len), dtype=np.int64)

                                if (dataset_type != 'DEBUG'):
                                    num_instances_check[dataset_type][setting] = np.zeros((num_folds[dataset_type], 3, len(idx_to_class_and_len)), dtype=np.int64)

                                for fold in range(num_folds[dataset_type]):
                                    # Start By sampling the classes that will be present in this fold.
                                    if (setting == 'N'):
                                        for split in splits:
                                            if (dataset_type == 'DEBUG'):
                                                classes_in_split[dataset_type][setting][fold][split] = list(seen_classes_debug)
                                            elif (dataset_type == 'HT'):
                                                classes_in_split[dataset_type][setting][fold][split] = list(seen_classes_as_idx)
                                            else:
                                                classes_in_split[dataset_type][setting][fold][split] = list(classes_as_idx)
                                    else:
                                        if (dataset_type == 'DEBUG'):
                                            sampled_classes = random.sample(seen_classes_debug, sum(num_classes[dataset_type][split] for split in splits))

                                            train_classes = sampled_classes[:num_classes[dataset_type]['train']]
                                            val_classes   = sampled_classes[num_classes[dataset_type]['train']:num_classes[dataset_type]['train']+num_classes[dataset_type]['val']]
                                            test_classes  = sampled_classes[-num_classes[dataset_type]['test']:]
                                        elif (dataset_type == 'final'):
                                            sampled_classes = random.sample(seen_classes_as_idx, sum(num_classes[dataset_type][split] for split in splits))

                                            train_classes = sampled_classes[:num_classes[dataset_type]['train']]
                                            val_classes   = sampled_classes[num_classes[dataset_type]['train']:num_classes[dataset_type]['train']+num_classes[dataset_type]['val']]
                                            test_classes  = sampled_classes[-num_classes[dataset_type]['test']:]
                                        else:
                                            train_classes = random.sample(seen_classes_as_idx, num_classes[dataset_type]['train'])
                                            sampled_unseen_classes = random.sample(unseen_classes_as_idx, num_classes[dataset_type]['val'] + num_classes[dataset_type]['test'])
                                            val_classes  = sampled_unseen_classes[:num_classes[dataset_type]['val']]
                                            test_classes = sampled_unseen_classes[num_classes[dataset_type]['val']:]

                                        classes_in_split[dataset_type][setting][fold]['train'] = train_classes
                                        classes_in_split[dataset_type][setting][fold]['val']   = val_classes
                                        classes_in_split[dataset_type][setting][fold]['test']  = test_classes


                                    # Now we sample the number of instances for this fold.
                                    if (setting == 'N' or setting == 'ZS'):
                                        for split in splits:
                                            if (split != 'train'):
                                                min_count = min_count_eval
                                            else:
                                                min_count = 1 if setting == 'N' else min_count_train
                                            samples = get_num_samples_and_guarantee_min_count(classes_num_insts[classes_in_split[dataset_type][setting][fold][split]],
                                                                                              split_size[dataset_type][split], min_count=min_count)
                                            num_instances_check[dataset_type][setting][fold, splits[split], classes_in_split[dataset_type][setting][fold][split]] = samples

                                    elif (setting == 'GZS'):
                                        for split in splits:
                                            min_count = min_count_train if split == 'train' else min_count_eval
                                            if (split == 'train'):
                                                samples = get_num_samples_and_guarantee_min_count(classes_num_insts[classes_in_split[dataset_type][setting][fold][split]],
                                                                                                  split_size[dataset_type][split], min_count=min_count)
                                                num_instances_check[dataset_type][setting][fold, splits[split], classes_in_split[dataset_type][setting][fold][split]] = samples
                                            else:
                                                t_samples = get_num_samples_and_guarantee_min_count(classes_num_insts[classes_in_split[dataset_type][setting][fold]['train']],
                                                                                                    int(0.5*split_size[dataset_type][split]), min_count=min_count_eval)
                                                e_samples = get_num_samples_and_guarantee_min_count(classes_num_insts[classes_in_split[dataset_type][setting][fold][split]],
                                                                                                    split_size[dataset_type][split] - int(0.5*split_size[dataset_type][split]),
                                                                                                    min_count=min_count_eval)
                                                num_instances_check[dataset_type][setting][fold, splits[split], classes_in_split[dataset_type][setting][fold]['train']] = t_samples
                                                num_instances_check[dataset_type][setting][fold, splits[split], classes_in_split[dataset_type][setting][fold][split]]   = e_samples

                                    else: # (Generalised) Few-Shot Setting.
                                        num_shots = int(setting.split('-')[1])
                                        t_samples_from_val = get_num_samples_and_guarantee_min_count(classes_num_insts[classes_in_split[dataset_type][setting][fold]['val']],
                                                                                                     num_shots*len(classes_in_split[dataset_type][setting][fold]['val']),
                                                                                                     min_count=num_shots)
                                        t_samples_from_tst = get_num_samples_and_guarantee_min_count(classes_num_insts[classes_in_split[dataset_type][setting][fold]['test']],
                                                                                                     num_shots*len(classes_in_split[dataset_type][setting][fold]['test']),
                                                                                                     min_count=num_shots)
                                        eval_num_samples = np.sum(t_samples_from_val) + np.sum(t_samples_from_tst)
                                        t_samples_from_trn = get_num_samples_and_guarantee_min_count(classes_num_insts[classes_in_split[dataset_type][setting][fold]['train']],
                                                                                                     split_size[dataset_type]['train'] - eval_num_samples,
                                                                                                     min_count=min_count_train)
                                        num_instances_check[dataset_type][setting][fold, splits['train'], classes_in_split[dataset_type][setting][fold]['train']] = t_samples_from_trn
                                        num_instances_check[dataset_type][setting][fold, splits['train'], classes_in_split[dataset_type][setting][fold]['val']]   = t_samples_from_val
                                        num_instances_check[dataset_type][setting][fold, splits['train'], classes_in_split[dataset_type][setting][fold]['test']]  = t_samples_from_tst

                                        for split in splits:
                                            if (split == 'train'):
                                                continue
                                            else:
                                                if (setting[0] == 'F'): # Non-Generalised Setting
                                                    samples = get_num_samples_and_guarantee_min_count(classes_num_insts[classes_in_split[dataset_type][setting][fold][split]],
                                                                                                      split_size[dataset_type][split], min_count=min_count_eval)
                                                    num_instances_check[dataset_type][setting][fold, splits[split], classes_in_split[dataset_type][setting][fold][split]] = samples

                                                else: # Generalised Setting
                                                    t_samples = get_num_samples_and_guarantee_min_count(classes_num_insts[classes_in_split[dataset_type][setting][fold]['train']],
                                                                                                        int(0.5*split_size[dataset_type][split]), min_count=min_count_eval)
                                                    e_samples = get_num_samples_and_guarantee_min_count(classes_num_insts[classes_in_split[dataset_type][setting][fold][split]],
                                                                                                        split_size[dataset_type][split] - int(0.5*split_size[dataset_type][split]),
                                                                                                        min_count=min_count_eval)
                                                    num_instances_check[dataset_type][setting][fold, splits[split], classes_in_split[dataset_type][setting][fold]['train']] = t_samples
                                                    num_instances_check[dataset_type][setting][fold, splits[split], classes_in_split[dataset_type][setting][fold][split]]   = e_samples


                                    # Verify that it is possible to build this fold.
                                    if (dataset_type == 'HT'):
                                        fold_num_insts = np.less_equal(np.sum(num_instances_check[dataset_type][setting][fold], axis=0), classes_num_insts)
                                    elif (dataset_type == 'final'):
                                        fold_train_num_insts = np.maximum(disjoint_train_val_HT + test_HT_maximums, num_instances_check[dataset_type][setting][fold][splits['train']])
                                        fold_num_insts = fold_train_num_insts + np.sum(num_instances_check[dataset_type][setting][fold][1:], axis=0)

                                    verify_fold_less_equal = np.less_equal(fold_num_insts, classes_num_insts)
                                    err_msg = dataset_type + ", " + setting + ", " + str(fold) + ": " + str(np.sum(verify_fold_less_equal))
                                    assert verify_fold_less_equal.all(), err_msg

                                    # Increment counts to test for setting disjointness.
                                    disjoint_train_setting += num_instances_check[dataset_type][setting][fold][splits['train']]
                                    disjoint_val_setting   += num_instances_check[dataset_type][setting][fold][splits['val']]
                                    if (test_only_in_other_tests):
                                        test_setting_maximums = np.maximum(test_setting_maximums, num_instances_check[dataset_type][setting][fold][splits['test']])

                                # Now verify that this entire setting is possible by itself.
                                if (test_only_in_other_tests):
                                    if (dataset_type == 'HT'):
                                        total_setting_num_instances = disjoint_train_setting + disjoint_val_setting + test_setting_maximums
                                    elif (dataset_type == 'final'):
                                        # This accounts for the train set being allowed to reuse instances from the 'HT' dataset.
                                        setting_train_num_insts = np.maximum(disjoint_train_val_HT + test_HT_maximums, disjoint_train_setting)
                                        total_setting_num_instances = setting_train_num_insts + disjoint_val_setting + test_setting_maximums
                                verify_setting_less_equal = np.less_equal(total_setting_num_instances, classes_num_insts)
                                
                                num_verified_classes = np.sum(verify_setting_less_equal)
                                if (max_verified_classes_so_far < num_verified_classes):
                                    max_verified_classes_so_far = num_verified_classes

                                err_msg = dataset_type + ", " + setting + ": " + str(np.sum(verify_setting_less_equal))
                                err_msg += " | " + str(num_verified_classes) + " (" + str(max_verified_classes_so_far) + ")"
                                assert verify_setting_less_equal.all(), err_msg

                                found_setting_solution = True

                            except AssertionError as err:
                                if (not found_setting_solution):
                                    if (dataset_type == 'HT'):
                                        if (num_tries_setting_solution == 500):
                                            raise AssertionError(err)
                                    elif (dataset_type == 'final'):
                                        if (num_tries_setting_solution == 10000):
                                            raise AssertionError(err)
                                    if ((num_tries_setting_solution - 1) % 50 == 0):
                                        print('\r' + str(err) + " | " + str(num_tries_setting_solution), end="", flush=True)
                                    num_tries_setting_solution += 1

                        print("\rFound solution for (" + dataset_type + ", " + setting + ") after " + str(num_tries_setting_solution) + " iterations.")

                        # Increment counts to test for 'HT' disjointness and check for disjointness.
                        if (dataset_type == 'HT'):
                            disjoint_train_val_HT += disjoint_train_setting + disjoint_val_setting
                            if (test_only_in_other_tests):
                                test_HT_maximums = np.maximum(test_HT_maximums, test_setting_maximums)

                            # Here we check if it is still possible (with this new setting) to create the dataset_type 'HT'.
                            verify_HT_settings_less_equal = np.less_equal(disjoint_train_val_HT + test_HT_maximums, classes_num_insts)
                            err_msg = dataset_type + " (verified number of settings before failure): " + str(setting_num) + "/" + str(len(settings)) + "."
                            err_msg += " | " + str(maximum_number_of_settings_verified)
                            assert verify_HT_settings_less_equal.all(), err_msg
                            maximum_number_of_settings_verified += 1

                            if (maximum_number_of_settings_verified == len(settings)):
                                found_dataset_type_solution = True

                except AssertionError as err:
                    if (not found_dataset_type_solution):
                        if (num_tries_dataset_type_solution == 500):
                            raise RuntimeError(err)
                        print('\r' + str(err) + " | " + str(num_tries_dataset_type_solution), end="", flush=True)
                        num_tries_dataset_type_solution += 1

                        clear_output(True)
                        print("TEST DISJOINTNESS TYPE:", "in other folds' train/val splits." if test_only_in_other_tests else "NOT in other folds' train/val splits.")
                        print()

            print("\rFound solution for (" + dataset_type + ") after " + str(num_tries_dataset_type_solution) + " iterations.")
            if (dataset_type == 'final'):
                found_global_solution = True

    except RuntimeError as err:
        if (not found_global_solution):
            num_tries_global_solution += 1
                                    
                                
                    
                    
                    
                    
                    
#             if (dataset_type == 'DEBUG'):
#                 seen_classes_debug = random.sample(seen_classes_as_idx, sum(num_classes[dataset_type][split] for split in splits))
#                 min_count_train = 1
#                 min_count_eval  = 1
#             else:
#                 if (dataset_type = 'HT'):
#                     disjoint_train_val_HT = np.zeros(len(idx_to_class_and_len), dtype=np.int64)
#                     if (test_only_in_other_tests):
#                         test_maximums_HT = np.zeros(len(idx_to_class_and_len), dtype=np.int64)
#                 min_count_train = 20
#                 min_count_eval  = 5
#             try:
#                 for setting in settings:
#                     num_setting_tries = 1
#                     max_verified_classes_so_far = 0
#                     found_setting_solution = False
#                     while (not found_setting_solution):
#                         try:
#                             if (dataset_type != 'DEBUG'):
#                                 num_instances_check[dataset_type][setting] = np.zeros((num_folds[dataset_type], 3, len(idx_to_class_and_len)), dtype=np.int64)
#         #                         disjoint_train = np.zeros(len(idx_to_class_and_len), dtype=np.int64)
#         #                         disjoint_val = np.zeros(len(idx_to_class_and_len), dtype=np.int64)
#         #                         disjoint_train_val = np.zeros(len(idx_to_class_and_len), dtype=np.int64)
#         #                         if (not test_can_be_in_other_fold_train_val_splits):
#         #                             test_maximums = np.zeros(len(idx_to_class_and_len), dtype=np.int64)

#                             for fold in range(num_folds[dataset_type]):

#                                 # Start By sampling the classes that will be present in this fold.
#                                 if (setting == 'N'):
#                                     for split in splits:
#                                         if (dataset_type == 'DEBUG'):
#                                             classes_in_split[dataset_type][setting][fold][split] = list(seen_classes_debug)
#                                         elif (dataset_type == 'HT'):
#                                             classes_in_split[dataset_type][setting][fold][split] = list(seen_classes_as_idx)
#                                         else:
#                                             classes_in_split[dataset_type][setting][fold][split] = list(classes_as_idx)
#                                 else:
#                                     if (dataset_type == 'DEBUG'):
#                                         sampled_classes = random.sample(seen_classes_debug, sum(num_classes[dataset_type][split] for split in splits))

#                                         train_classes = sampled_classes[:num_classes[dataset_type]['train']]
#                                         val_classes   = sampled_classes[num_classes[dataset_type]['train']:num_classes[dataset_type]['train']+num_classes[dataset_type]['val']]
#                                         test_classes  = sampled_classes[-num_classes[dataset_type]['test']:]
#                                     elif (dataset_type == 'final'):
#                                         sampled_classes = random.sample(seen_classes_as_idx, sum(num_classes[dataset_type][split] for split in splits))

#                                         train_classes = sampled_classes[:num_classes[dataset_type]['train']]
#                                         val_classes   = sampled_classes[num_classes[dataset_type]['train']:num_classes[dataset_type]['train']+num_classes[dataset_type]['val']]
#                                         test_classes  = sampled_classes[-num_classes[dataset_type]['test']:]
#                                     else:
#                                         train_classes = random.sample(seen_classes_as_idx, num_classes[dataset_type]['train'])
#                                         sampled_unseen_classes = random.sample(unseen_classes_as_idx, num_classes[dataset_type]['val'] + num_classes[dataset_type]['test'])
#                                         val_classes  = sampled_unseen_classes[:num_classes[dataset_type]['val']]
#                                         test_classes = sampled_unseen_classes[num_classes[dataset_type]['val']:]

#                                     classes_in_split[dataset_type][setting][fold]['train'] = train_classes
#                                     classes_in_split[dataset_type][setting][fold]['val']   = val_classes
#                                     classes_in_split[dataset_type][setting][fold]['test']  = test_classes


#                                 # Now we sample the number of instances for this fold.
#                                 if (setting == 'N' or setting == 'ZS'):
#                                     for split in splits:
#                                         if (split != 'train'):
#                                             min_count = min_count_eval
#                                         else:
#                                             min_count = 1 if setting == 'N' else min_count_train
#                                         samples = get_num_samples_and_guarantee_min_count(classes_num_insts[classes_in_split[dataset_type][setting][fold][split]],
#                                                                                           split_size[dataset_type][split], min_count=min_count)
#                                         num_instances_check[dataset_type][setting][fold, splits[split], classes_in_split[dataset_type][setting][fold][split]] = samples

#                                 elif (setting == 'GZS'):
#                                     for split in splits:
#                                         min_count = min_count_train if split == 'train' else min_count_eval
#                                         if (split == 'train'):
#                                             samples = get_num_samples_and_guarantee_min_count(classes_num_insts[classes_in_split[dataset_type][setting][fold][split]],
#                                                                                               split_size[dataset_type][split], min_count=min_count)
#                                             num_instances_check[dataset_type][setting][fold, splits[split], classes_in_split[dataset_type][setting][fold][split]] = samples
#                                         else:
#                                             t_samples = get_num_samples_and_guarantee_min_count(classes_num_insts[classes_in_split[dataset_type][setting][fold]['train']],
#                                                                                                 int(0.5*split_size[dataset_type][split]), min_count=min_count_eval)
#                                             e_samples = get_num_samples_and_guarantee_min_count(classes_num_insts[classes_in_split[dataset_type][setting][fold][split]],
#                                                                                                 split_size[dataset_type][split] - int(0.5*split_size[dataset_type][split]),
#                                                                                                 min_count=min_count_eval)
#                                             num_instances_check[dataset_type][setting][fold, splits[split], classes_in_split[dataset_type][setting][fold]['train']] = t_samples
#                                             num_instances_check[dataset_type][setting][fold, splits[split], classes_in_split[dataset_type][setting][fold][split]]   = e_samples

#                                 else: # (Generalised) Few-Shot Setting.
#                                     num_shots = int(setting.split('-')[1])
#                                     t_samples_from_val = get_num_samples_and_guarantee_min_count(classes_num_insts[classes_in_split[dataset_type][setting][fold]['val']],
#                                                                                                  num_shots*len(classes_in_split[dataset_type][setting][fold]['val']),
#                                                                                                  min_count=num_shots)
#                                     t_samples_from_tst = get_num_samples_and_guarantee_min_count(classes_num_insts[classes_in_split[dataset_type][setting][fold]['test']],
#                                                                                                  num_shots*len(classes_in_split[dataset_type][setting][fold]['test']),
#                                                                                                  min_count=num_shots)
#                                     eval_num_samples = np.sum(t_samples_from_val) + np.sum(t_samples_from_tst)
#                                     t_samples_from_trn = get_num_samples_and_guarantee_min_count(classes_num_insts[classes_in_split[dataset_type][setting][fold]['train']],
#                                                                                                  split_size[dataset_type]['train'] - eval_num_samples,
#                                                                                                  min_count=min_count_train)
#                                     num_instances_check[dataset_type][setting][fold, splits['train'], classes_in_split[dataset_type][setting][fold]['train']] = t_samples_from_trn
#                                     num_instances_check[dataset_type][setting][fold, splits['train'], classes_in_split[dataset_type][setting][fold]['val']]   = t_samples_from_val
#                                     num_instances_check[dataset_type][setting][fold, splits['train'], classes_in_split[dataset_type][setting][fold]['test']]  = t_samples_from_tst

#                                     for split in splits:
#                                         if (split == 'train'):
#                                             continue
#                                         else:
#                                             if (setting[0] == 'F'): # Non-Generalised Setting
#                                                 samples = get_num_samples_and_guarantee_min_count(classes_num_insts[classes_in_split[dataset_type][setting][fold][split]],
#                                                                                                   split_size[dataset_type][split], min_count=min_count_eval)
#                                                 num_instances_check[dataset_type][setting][fold, splits[split], classes_in_split[dataset_type][setting][fold][split]] = samples

#                                             else: # Generalised Setting
#                                                 t_samples = get_num_samples_and_guarantee_min_count(classes_num_insts[classes_in_split[dataset_type][setting][fold]['train']],
#                                                                                                     int(0.5*split_size[dataset_type][split]), min_count=min_count_eval)
#                                                 e_samples = get_num_samples_and_guarantee_min_count(classes_num_insts[classes_in_split[dataset_type][setting][fold][split]],
#                                                                                                     split_size[dataset_type][split] - int(0.5*split_size[dataset_type][split]),
#                                                                                                     min_count=min_count_eval)
#                                                 num_instances_check[dataset_type][setting][fold, splits[split], classes_in_split[dataset_type][setting][fold]['train']] = t_samples
#                                                 num_instances_check[dataset_type][setting][fold, splits[split], classes_in_split[dataset_type][setting][fold][split]]   = e_samples

#                                 if (dataset_type == 'HT'):
#                                     disjoint_train_val_HT += num_instances_check[dataset_type][setting][fold][:2]
#                                     if (test_only_in_other_tests):
#                                         test_maximums_HT = np.maximum(test_maximums_HT, num_instances_check[dataset_type][setting][fold][splits['test']])


#         #                         if (not test_can_be_in_other_fold_train_val_splits):
#         #                             test_maximums = np.maximum(test_maximums, num_instances_check[dataset_type][setting][fold][splits['test']])

#         #                         disjoint_train += num_instances_check[dataset_type][setting][fold][splits['train']]
#         #                         disjoint_val   += num_instances_check[dataset_type][setting][fold][splits['val']]
#         # #                         disjoint_train_val += np.sum(num_instances_check[dataset_type][setting][fold][:2], axis=0)
#                                 verify_fold_less_equal = np.less_equal(np.sum(num_instances_check[dataset_type][setting][fold], axis=0), classes_num_insts)
#                                 err_msg = dataset_type + ", " + setting + ", " + str(fold) + ": " + str(np.sum(verify_fold_less_equal))
#                                 assert verify_fold_less_equal.all(), err_msg

#         #                     if (test_can_be_in_other_fold_train_val_splits):
#         #                         if (dataset_type = 'HT'):
#         #                             disjoint_HT_setting_maximums = np.zeros(len(idx_to_class_and_len), dtype=np.int64)
#         #                         for fold in range(num_folds[dataset_type]):
#         #                             train_val_values = np.sum(num_instances_check[dataset_type][setting][fold][:2], axis=0)
#         #                             other_train_val_values = disjoint_train_val - train_val_values
#         #                             test_in_other_train_val_splits = np.maximum(other_train_val_values, num_instances_check[dataset_type][setting][fold][splits['test']])
#         #                             if (dataset_type = 'HT'):
#         #                                 disjoint_HT_setting_maximums = np.maximum(disjoint_HT_setting_maximums, test_in_other_train_val_splits)
#         #                             verify_setting_disjointness = np.less_equal(train_val_values + test_in_other_train_val_splits, classes_num_insts)
#         #                             err_msg = "\rSETTING DISJOINTNESS (across fold): " + dataset_type + ", " + setting + ", " + str(fold) + ": "
#         #                             if (not verify_setting_disjointness.all()):
#         #                                 break
#         #                         if (dataset_type = 'HT'):
#         #                             disjoint_HT += disjoint_HT_setting_maximums
#         #                     else:
#         #                         if (dataset_type = 'HT'):
#         #                             disjoint_HT += disjoint_train_val + test_maximums
#         #                         verify_setting_disjointness = np.less_equal(disjoint_train_val + test_maximums, classes_num_insts)
#         #                         err_msg = "\rSETTING DISJOINTNESS: " + dataset_type + ", " + setting + ", (Test not in train/val of other folds): "

#         #                     # Here we check for the required disjointness aspects.
#         #                     if (dataset_type == 'final'):
#         #                         disjointness_with_HT = np.maximum(disjoint_HT, )
#         #                     num_verified_classes = np.sum(verify_setting_disjointness)
#         #                     if (num_verified_classes > max_verified_classes_so_far):
#         #                         max_verified_classes_so_far = num_verified_classes

#         #                     err_msg += str(np.sum(verify_setting_disjointness)) + " (max: " + str(max_verified_classes_so_far) + ")"
#         #                     assert verify_setting_disjointness.all(), err_msg

#         #                     # Check that all classes are present in this setting. TODO: ADAPT PROPERLY FOR DIFFERENT dataset_type
#         #                     classes_present_in_split = np.sum(np_sum(num_instances_check[dataset_type][setting], axis=1), axis=0) != 0
#         #                     err_msg = "\rASSERT ALL CLASSES IN SETTING: " + dataset_type + ", " + setting + " | Num classes in setting:  "
#         #                     err_msg += str(np.sum(classes_present_in_split)) + "."
#         #                     assert (classes_present_in_split).all()

#         #                     found_setting_solution = True

#                         except AssertionError as err:
#                             if (not found_setting_solution):
#                                 if ((num_setting_tries - 1) % 50 == 0):
#                                     print(err, end="", flush=True)
#                                 num_setting_tries += 1

#                     print("\rFound solution for (" + dataset_type + ", " + setting + ") after " + str(num_setting_tries) + " iterations.")
#                 # TODO: Check for disjointness for 'HT' dataset
#             if (dataset_type == 'HT'):
#                 verify_fold_less_equal = np.less_equal(np.sum(num_instances_check[dataset_type][setting][fold], axis=0), classes_num_insts)
#                 err_msg = dataset_type + ", " + setting + ", " + str(fold) + ": " + str(np.sum(verify_fold_less_equal))
#                 assert verify_fold_less_equal.all(), err_msg

#             except AssertionError as err:
#                 raise AssertionError(err)# from None


np.set_printoptions(formatter={'all':lambda x: '{:5d}'.format(x)})
def specific_dataset_print(dataset_type, setting, fold, split, mask_train):
    # The interact module automatically retrieves the dictionary values, for the corresponding keys.

    print("\n"*5)
    if (setting != 'N'):
        train_mask = (num_instances_check[dataset_type][setting][fold][0] == 0).astype(np.int8)
        if (setting[:3] == 'GFS' or setting[:3] == 'FS-'):
            train_mask += (num_instances_check[dataset_type][setting][fold][0] == int(setting.split('-')[1])).astype(np.int8)
    else:
        train_mask = np.ones_like(num_instances_check[dataset_type][setting][fold][0], dtype=np.int8)

    if (setting[:3] == 'GFS' or setting[:3] == 'FS-'):
        anti_train_mask = (~train_mask.astype(np.bool)).astype(np.int8)
    else:
        anti_train_mask = np.ones_like(num_instances_check[dataset_type][setting][fold][0], dtype=np.int8)
        

    masked = np.multiply(train_mask, num_instances_check[dataset_type][setting][fold][split])
    anti_masked = np.multiply(anti_train_mask, num_instances_check[dataset_type][setting][fold][split])

    print('Fold num insts:', sum(np.sum(num_instances_check[dataset_type][setting][fold][split_num]) for split_num in [0, 1, 2]))
    print('Split num insts:', np.sum(num_instances_check[dataset_type][setting][fold][split]))
    print("'Exclusive' split num insts:", np.sum(masked if split != 0 else anti_masked))
    print()
    print(masked if mask_train else num_instances_check[dataset_type][setting][fold][split])

def select_dataset_type_print(dataset_type):
    global settings
    global num_folds
    global splits
    interact(specific_dataset_print, dataset_type=fixed(dataset_type), setting=settings,
             fold=list(range(num_folds[dataset_type])), split=splits, mask_train=False)

interact(select_dataset_type_print, dataset_type=dataset_types);

TEST DISJOINTNESS TYPE: in other folds' train/val splits.

Found solution for (HT, N) after 1 iterations.


KeyboardInterrupt: 

In [129]:
np.set_printoptions()

In [62]:
# Start with the Hyperparameter dataset type.
dataset_type = 'final'

for setting in settings:
    
    found_setting_solution = False
    num_its = 1
    while (not found_setting_solution):
        try:
            num_instances_check[dataset_type][setting] = np.zeros((num_folds['final'], 3, len(idx_to_class_and_len)), dtype=np.int64)
            train_val_disjoint = np.zeros(len(idx_to_class_and_len), dtype=np.int64)
            for fold in range(num_instances_check[dataset_type][setting].shape[0]):
                for split in splits:
                    samples = get_num_samples_and_guarantee_min_count(classes_num_insts[classes_in_split[dataset_type][setting][fold][split]],
                                                                      split_size[dataset_type][split], min_count=1)
                    num_instances_check[dataset_type][setting][fold, splits[split], classes_in_split[dataset_type][setting][fold][split]] = samples

                    if (split != 'test'):
                        train_val_disjoint += num_instances_check[dataset_type][setting][fold][splits[split]]


                # Verify fold disjointness
                verify_less_equal = np.less_equal(np.sum(num_instances_check[dataset_type][setting][fold], axis=0), classes_num_insts)
                err_msg = dataset_type + ", " + setting + ", " + str(fold) + ": " + str(np.sum(verify_less_equal))
                assert verify_less_equal.all(), err_msg

            # Verify setting train/val disjointness
            for fold in range(num_instances_check[dataset_type][setting].shape[0]):
                test_values = np.zeros(len(idx_to_class_and_len), dtype=np.int64)
                test_values += num_instances_check[dataset_type][setting][fold][splits['test']]
                train_val_values = np.zeros(len(idx_to_class_and_len), dtype=np.int64)
                train_val_values += num_instances_check[dataset_type][setting][fold][splits['train']]
                train_val_values += num_instances_check[dataset_type][setting][fold][splits['val']]
                other_train_val_values = train_val_disjoint - train_val_values
                test_in_other_train_val_splits = np.maximum(other_train_val_values, test_values)
                verify_less_equal = np.less_equal(train_val_values + test_in_other_train_val_splits, classes_num_insts)
                err_msg = "SETTING DISJOINTNESS: " + dataset_type + ", " + setting + ", " + str(fold) + ": " + str(np.sum(verify_less_equal))
                assert verify_less_equal.all(), err_msg

            found_setting_solution = True
            print("\nFound solution for dataset_type: " + dataset_type + ", for setting: " + setting + ", after " + str(num_its) + " iterations.")
        except AssertionError as err:
            num_its += 1
            print(train_val_values + test_in_other_train_val_splits, classes_num_insts, verify_less_equal)
            print('\r', err, end="", flush=True)
            pass


Found solution for dataset_type: final, for setting: N, after 1 iterations.

Found solution for dataset_type: final, for setting: ZS, after 1 iterations.

Found solution for dataset_type: final, for setting: GZS, after 1 iterations.

Found solution for dataset_type: final, for setting: FS-1, after 1 iterations.
[76773 75984 52176 42101 34062 28670 18882 17484 15773 13913 10235  9639
  8586  8085  7187  6922  6227  5785  5612  5136  5025  3527  3089  2561
  2505  2466  2198  1926  1687  1713  1677  1559  1489  1482  1322  1264
  1178  1075  1055  1074  1038   925   940   892   681   579   574   533
   532   449   449   441   403   428   363   367   320   327   365   290
   473   259   307   457   327   236   288   220   220   204   148   383
   386   228   202   591   158   288   159   372   118   154   122    64
   138   263   272   228   102   338    52    89    68   167   100   108
   196   102    85    79    85   176   135    59    86    92    55    46
   103    66    60    39    4

KeyboardInterrupt: 

In [30]:
print(num_instances_check['final']['ZS'][0])

[[7718 7457 5188 4109 3439 2779 1977 1689 1638 1366 1009 1012  846  758
   746  698  638  589  596  476  456  363  330  282  263  299  237  195
   175  167  190  187  160  165  140  128   97  102  104  113   92   92
    81   75   66   51   58   63   63   57   49   48   52   54   34   38
    37   38   38   33    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0   37    0   51
     0    0    0   43

In [23]:
num_instances = 2314975
print(num_instances-836239)

1478736


In [7]:
from allennlp.data.tokenizers.word_filter import StopwordFilter
stop_words_filter = StopwordFilter(tokens_to_add=['SUBJECT_ENTITY'.lower(), 'OBJECT_ENTITY'.lower()])

In [20]:
# Load the information regarding all relation descriptions.
with open(path_datasets + 'Ours/ELMO_rel_descs_idxs.map', 'rb') as f:
    idxs_of_relation = pickle.load(f)
    rel_desc_to_idx  = pickle.load(f)
    idx_to_rel_desc  = pickle.load(f)


# Load the relations in the train file.
relations = []
with open(path_ours + "run_time_testing/normal/train_relations.txt", 'r', encoding='utf-8') as f:
    for line in f:
        relations.append(line.strip())


# Get the idxs of the relations that are present in the train split.
idxs_of_relations_in_train = list(set(idx for relation in relations for idx in idxs_of_relation[relation]))
idxs_of_relations_in_train.sort() # We don't really need sort, but we do just for consistency.


# Load the information regarding the tokens present in the train split.
with open(path_ours + "run_time_testing/normal/train.ulbs", 'rb') as f:
    _                = pickle.load(f)
    idx_to_token_map = pickle.load(f)
    token_to_idx_map = pickle.load(f) # We don't need this, but we use it for debugging.



# Get the unsupervised labels for the tokens of the relations' descriptions. We only get those tokens that are present in the training vocabulary.
rels_descs_idxs_in_train_to_ulbls = [None]*len(idxs_of_relations_in_train)
for i, rel_idx in enumerate(idxs_of_relations_in_train):
    ulbls = []
    rel_desc = tokenizer.tokenize(idx_to_rel_desc[rel_idx])
    for token in [word.text for word in rel_desc]:
        if token in [word.text for word in stop_words_filter.filter_words(rel_desc)] and token in token_to_idx_map:
            ulbls.append(token_to_idx_map[token])
        else:
            ulbls.append(0)

    if (any(token_id != 0 for token_id in ulbls)):
        rels_descs_idxs_in_train_to_ulbls[i] = ulbls



idxs_of_relations_in_train = [idx for i, idx in enumerate(idxs_of_relations_in_train) if rels_descs_idxs_in_train_to_ulbls[i] is not None]
rels_descs_idxs_in_train_to_ulbls = [ulbls for ulbls in rels_descs_idxs_in_train_to_ulbls if ulbls is not None]

with open(path_ours + "run_time_testing/normal/rel_descs_basic.ulbs", 'wb+') as f:
    pickle.dump(idxs_of_relations_in_train, f)
    pickle.dump(rels_descs_idxs_in_train_to_ulbls, f)

print("DONE")
# print(len(idxs_of_relations_in_train))
# print(len(rels_descs_idxs_in_train_to_ulbls))
# print(idxs_of_relations_in_train)
# print(rels_descs_idxs_in_train_to_ulbls)

DONE


In [43]:
a = 'I went to the store OBJECT_ENTITY the other day.'
a = stop_words_filter.filter_words(tokenizer.tokenize(a))
print(a)

# Load the relations in the train file.
relations = []
with open(path_ours + "run_time_testing/normal/train_relations.txt", 'r', encoding='utf-8') as f:
    for line in f:
        relations.append(line.strip())

# Load the relations' descriptions.
rels_descs = {}
for relation in relations:
    rels_descs[relation] = []
    file_name_rels_descs = path_ours + 'relations_descriptions/' + re.sub('/', '-', re.sub(' ', '_', relation)) + '.txt'
    with open(file_name_rels_descs, 'r', encoding='utf-8') as f:
        for line in f:
            rels_descs[relation].append(tokenizer.tokenize(line.strip()))


with open(path_ours + "run_time_testing/normal/train.ulbs", 'rb') as f:
    data_as_idxs     = pickle.load(f)
    idx_to_token_map = pickle.load(f)
    token_to_idx_map = pickle.load(f)


#[word.text for word in stop_words_filter.filter_words(rel_desc)]
unseen_tokens = set()
rels_descs_ulbls = {}
for relation in rels_descs:
    rels_descs_ulbls[relation] = []
    for rel_desc in rels_descs[relation]:
        rel_desc_ulbls = []
        for token in [word.text for word in rel_desc]:
            if token in [word.text for word in stop_words_filter.filter_words(rel_desc)] and token in token_to_idx_map:
                rel_desc_ulbls.append(token_to_idx_map[token])
            else:
                rel_desc_ulbls.append(0)
                unseen_tokens.add(token)
        rels_descs_ulbls[relation].append(rel_desc_ulbls)


print(sum(sum(all(label == 0 for label in rel_desc_ulbls) for rel_desc_ulbls in rels_descs_ulbls[relation]) == len(rels_descs[relation]) for relation in rels_descs))
print(sum(sum(all(label == 0 for label in rel_desc_ulbls) for rel_desc_ulbls in rels_descs_ulbls[relation]) for relation in rels_descs))
print(rels_descs_ulbls['license'])
print(idx_to_token_map[57-1])
for relation in rels_descs:
    if (sum(all(label == 0 for label in rel_desc_ulbls) for rel_desc_ulbls in rels_descs_ulbls[relation]) > 0):
        print(relation, len(rels_descs[relation]), sum(all(label == 0 for label in rel_desc_ulbls) for rel_desc_ulbls in rels_descs_ulbls[relation]))
# print(sum(all(label == 0 for label in rel_desc_ulbls) for rel_desc_ulbls in rels_descs_ulbls))
# print("NUM UNSEEN TOKENS:", len(unseen_tokens))

# print(rels_descs_ulbls)

[went, store, other, day]
1
57
[[0, 0, 0, 0, 0, 0]]
based
architect 29 2
author 27 3
based on 2 1
child 2 1
chromosome 10 1
continent 18 7
country of citizenship 16 1
creator 7 2
date of official opening 14 1
director 23 1
discoverer or inventor 15 2
distributor 19 4
editor 4 1
father 14 3
film editor 17 1
head of government 4 1
languages spoken or written 8 3
license 1 1
lyrics by 12 1
native language 10 4
original network 34 1
participant of 9 2
performer 29 2
point in time 12 1
product 6 4
publication date 28 1
sex or gender 20 5


#### Create a file with the relations' descriptions, from which ELMO will create embeddings

In [24]:
# Total number of relation descriptions (when accounting for a description being associated for more than 1 relation)
print(sum(file_len(path_datasets + 'Ours/relations_descriptions/' + re.sub('/', '-', re.sub(' ', '_', relation)) + '.txt') for relation in relation_filepointers))

1135


In [25]:
rel_desc_to_idx  = {}
idx_to_rel_desc  = []
idxs_of_relation = {}

r_idx = 0

for r_num, relation in enumerate(relation_filepointers):
    file_name_positive = path_datasets + 'Ours/relations_descriptions/' + re.sub('/', '-', re.sub(' ', '_', relation)) + '.txt'
    with open(file_name_positive, 'r', encoding='utf-8') as f:
        for line in f:
            line_temp = line.strip()

            if (line_temp not in rel_desc_to_idx):
                rel_desc_to_idx[line_temp] = r_idx
                idx_to_rel_desc.append(line_temp)
                r_idx += 1

            if (relation not in idxs_of_relation):
                idxs_of_relation[relation] = [rel_desc_to_idx[line_temp]]
            else:
                idxs_of_relation[relation].append(rel_desc_to_idx[line_temp])



path_ELMO_relations = path_datasets + 'Ours/ELMO_relations.txt'

with open(path_ELMO_relations, 'w+', encoding='utf-8') as elmo_rel_file:
    for rel_desc in rel_desc_to_idx:
        elmo_rel_file.write(''.join(space_word_tokenize_string(rel_desc, tokenizer)) + '\n')


with open(path_datasets + 'Ours/ELMO_rel_descs_idxs.map', 'wb+') as f:
    pickle.dump(idxs_of_relation, f)
    pickle.dump(rel_desc_to_idx, f)
    pickle.dump(idx_to_rel_desc, f)

#### Create a single fold for the normal setting to evaluate how long things take to train/evaluate.

In [201]:
##################################################################################################################################################################
##################################################################################################################################################################
#                           ##############################################################################################                                       #
#                           #                                     ANY-SHOT QUESTIONS                                     #                                       #
#                           ##############################################################################################                                       #
#                                                                                                                                                                #
# GENERALISED ANY-SHOT                                                                                                                                           #
# 1. At evaluation time (so, for validation and test splits), what should be the distribution of elements of seen vs unseen classes? Would you set it as 50% for #
# seen and 50% for unseen (i.e., let's assume a split with 1000 instances: 500 would belong to the seen classes and 500 to the unseen classes)? Would you        #
# instead follow the data distribution and just guarantee that each class gets at least one observation? A mix of both (the 500 instances, in the previous       #
# example) would follow the data distribution of just the seen/unseen classes, correspondingly)?                                                                 #
#                                                                                                                                                                #
# (GENERALISED) FEW-SHOT                                                                                                                                         #
# 1. Is it a problem if there are more training instances than evaluation instances? This arises due to the few-shot condition of having a specific number of    #
# training instances, which might not leave enough data, for the classes with fewer instances, to evaluate on a large enough sample size. What would be an       #
# acceptable minimum number of instances at evaluation time?                                                                                                     #
#                                                                                                                                                                #
# 2. Is it necessary to also guarantee that the training classes comply with having at least the number of shot instances (I'm assuming so for Generalised       #
# Few-Shot Learning)? For my evaluation I am creating disjoint folds where I randomly sample the classes that will be in each split, which means some of the     #
# classes with the fewer instances might be assigned to the train split and, since I then sample instances according to the data distribution, those classes     #
# might get very few instances.                                                                                                                                  #
#                                                                                                                                                                #
##################################################################################################################################################################
##################################################################################################################################################################


# Maximum length of the x sentences (observations).
max_x_len = 60


balanced_train = False
balanced_eval  = False


# We define it so that both the validation and test splits have the same number of classes (whatever
# remains after deciding the number of classes in the train split). If the remaining number of classes
# is not divisible by 2, the test split gets the extra class. TODO: can easily extend this whole procedure for different number of train classes.
few_zero_shot_train_classes_num = 80
few_zero_shot_val_classes_num   = (len(relation_filepointers) - few_zero_shot_train_classes_num)//2
few_zero_shot_test_classes_num  = len(relation_filepointers) - few_zero_shot_train_classes_num - few_zero_shot_val_classes_num
few_shot_shot_nums = [1, 2, 5, 10]
generalised_few_shot_nums = [1, 2, 5]

num_classes = [[120, 120, 120], [few_zero_shot_train_classes_num, few_zero_shot_val_classes_num, few_zero_shot_test_classes_num]]

generalised_settings_percent_seen_classes = 0.5

evaluation_settings = ['N', 'ZS', 'GZS']
evaluation_settings += ['FS-' + str(n) for n in few_shot_shot_nums]
evaluation_settings += ['GFS-' + str(n) for n in generalised_few_shot_nums]
print(evaluation_settings)

splits = ['train', 'val', 'test']



#########################
# HYPERPARAMETER TUNING #
#########################

# Number of Sentences Per Split
train_HT = 10000
val_HT   = 500
test_HT  = 2500

# Number of folds, per setting.
K_folds_HT = 2


####################
# FINAL EVALUATION #
####################

# Number of Sentences Per Split
train_FE = 100000
val_FE   = 5000
test_FE  = 25000

# Number of folds, per setting.
K_folds_FE = 5


num_insts = {'hyperparameter': {'train': train_HT, 'val': val_HT, 'test': test_HT}, 'final': {'train': train_FE, 'val': val_FE, 'test': test_FE}}
K_folds   = {'hyperparameter': K_folds_HT, 'final': K_folds_FE}



#############################
# CONCERNING SPLITS OVERLAP #
#############################
# We consider that the 'val' & 'test' 'FINAL EVALUATION' ('FE') splits need to be completely disjoint
# from any previously seen data and disjoint between themselves (at the same fold level) and between
# folds for the validation splits. Train splits are also disjoint (also across folds).

                                      # THESE 'Done!' REFER ONLY TO ZERO-SHOT
disjoint_train_and_val_splits = True  # Done!
completely_disjoint_folds     = False # Done!

disjoint_settings_HT = True # If the datasets used for hyperparameter tuning should be disjoint also across settings.

assert sum([completely_disjoint_folds, disjoint_train_and_val_splits]) == 1, "Can only select one disjoint mode, not multiple or 0."


##################
# EARLY CHECKING #
##################

min_number_of_insts_per_setting_per_fold = {'N': 3, 'ZS': 1, 'GZS': 1}
# TODO: Do all the training classes also need to comply with the number of shots? or Can they have fewer instances?
min_number_of_insts_per_setting_per_fold.update({'FS-' + str(n): 1 for n in few_shot_shot_nums})
min_number_of_insts_per_setting_per_fold.update({'GFS-' + str(n): n + 1 for n in generalised_few_shot_nums})

min_number_of_insts_per_setting_per_fold_val_test = {'N': 2, 'ZS': 1, 'GZS': 1}
# TODO: Do all the training classes also need to comply with the number of shots? or Can they have fewer instances?
min_number_of_insts_per_setting_per_fold_val_test.update({'FS-' + str(n): 1 for n in few_shot_shot_nums})
min_number_of_insts_per_setting_per_fold_val_test.update({'GFS-' + str(n): 1 for n in generalised_few_shot_nums})

min_number_of_insts_per_setting_per_fold_train = {'N': 1, 'ZS': 0, 'GZS': 0}
# TODO: Do all the training classes also need to comply with the number of shots? or Can they have fewer instances?
min_number_of_insts_per_setting_per_fold_train.update({'FS-' + str(n): 1 for n in few_shot_shot_nums})
min_number_of_insts_per_setting_per_fold_train.update({'GFS-' + str(n): 1 for n in generalised_few_shot_nums})


HT_num_insts         = sum(K_folds_HT*min_number_of_insts_per_setting_per_fold[key] for key in min_number_of_insts_per_setting_per_fold)
HT_num_insts_train   = sum(K_folds_FE*min_number_of_insts_per_setting_per_fold_train[key] for key in min_number_of_insts_per_setting_per_fold_train)
FE_num_insts_val_tst = max(min_number_of_insts_per_setting_per_fold_val_test[key] for key in min_number_of_insts_per_setting_per_fold_val_test)

err_msg = "The class with the fewest instances does not have enough instances to find a configuration, with the current chosen parameters."
# TODO: Look into this when 'disjoint_settings_HT = False'
# assert HT_num_insts + FE_num_insts <= np.min(file_lens), err_msg + " " + str(np.min(file_lens)) + ", " + str(HT_num_insts) + ", " + str(FE_num_insts)


############################################################################################
# Try to find a number of samples per class distribution that fits the constraints imposed #
############################################################################################

classes = [class_to_idx[_class] for _class in class_to_idx]

no_solution_yet = True
num_tries = 0


while (no_solution_yet):
    try:
        counts = {}
        classes_in_split = {}
        for dataset_type in ['hyperparameter', 'final']:
            counts[dataset_type] = {}
            classes_in_split[dataset_type] = {}

            for setting in evaluation_settings:
                counts[dataset_type][setting] = {}
                classes_in_split[dataset_type][setting] = {}

                for split in splits:
                    counts[dataset_type][setting][split] = np.zeros([K_folds[dataset_type], len(relation_filepointers)]).astype(np.int64)

                num_classes_setting = 0 if setting == 'N' else 1
                trn_clss = num_classes[num_classes_setting][0]
                val_clss = num_classes[num_classes_setting][1]
                # Test classes can be inferred from the other 2.

                for fold_num in range(K_folds[dataset_type]):
                    classes_in_split[dataset_type][setting][fold_num] = {}

                    random.shuffle(classes)

                    val_classes  = None
                    test_classes = None

                    # Normal setting.
                    if (setting == 'N'):
                        train_classes = classes
                        val_classes   = classes
                        test_classes  = classes

                        trn = get_num_samples_and_guarantee_min_count(file_lens[train_classes], num_insts[dataset_type]['train'], 1)
                        val = get_num_samples_and_guarantee_min_count(file_lens[val_classes].astype(float), num_insts[dataset_type]['val'], 1)
                        tst = get_num_samples_and_guarantee_min_count(file_lens[test_classes].astype(float), num_insts[dataset_type]['test'], 1)

                        counts[dataset_type][setting]['train'][fold_num][train_classes] = trn
                        counts[dataset_type][setting]['val'][fold_num][val_classes]     = val
                        counts[dataset_type][setting]['test'][fold_num][test_classes]   = tst


                    # Zero-shot settings.
                    elif (setting == 'ZS' or setting == 'GZS'):
                        train_classes = classes[:trn_clss]
                        trn = get_num_samples_and_guarantee_min_count(file_lens[train_classes].astype(float), num_insts[dataset_type]['train'], 1)
                        counts[dataset_type][setting]['train'][fold_num][train_classes] = trn

                        if (setting == 'ZS'):
                            val_classes = classes[trn_clss:trn_clss+val_clss]
                            val = get_num_samples_and_guarantee_min_count(file_lens[val_classes].astype(float), num_insts[dataset_type]['val'], 1)
                            counts[dataset_type][setting]['val'][fold_num][val_classes] = val

                            test_classes = classes[trn_clss+val_clss:]
                            tst = get_num_samples_and_guarantee_min_count(file_lens[test_classes].astype(float), num_insts[dataset_type]['test'], 1)
                            counts[dataset_type][setting]['test'][fold_num][test_classes] = tst

                        else:
                            # *** Validation Split *** #
                            # First determine the number of instances that will belong to the seen classes and the number of instances that will belong to the unseen ones.
                            val_train_num_insts = int(generalised_settings_percent_seen_classes*num_insts[dataset_type]['val'])
                            val_val_num_insts   = num_insts[dataset_type]['val'] - val_train_num_insts

                            # Start by sampling the seen classes for the validation.
                            val_train_classes = classes[:trn_clss]
                            val_val_classes   = classes[trn_clss:trn_clss+val_clss]
                            val_train = get_num_samples_and_guarantee_min_count(file_lens[val_train_classes].astype(float), val_train_num_insts, 1)
                            val_val   = get_num_samples_and_guarantee_min_count(file_lens[val_val_classes].astype(float), val_val_num_insts, 1)
                            counts[dataset_type][setting]['val'][fold_num][val_train_classes] = val_train
                            counts[dataset_type][setting]['val'][fold_num][val_val_classes]   = val_val

                            # *** Test Split *** #
                            # First determine the number of instances that will belong to the seen classes and the number of instances that will belong to the unseen ones.
                            tst_train_num_insts = int(generalised_settings_percent_seen_classes*num_insts[dataset_type]['test'])
                            tst_tst_num_insts   = num_insts[dataset_type]['test'] - tst_train_num_insts

                            # Start by sampling the seen classes for the validation.
                            tst_train_classes = classes[:trn_clss]
                            tst_tst_classes   = classes[trn_clss+val_clss:]
                            tst_train = get_num_samples_and_guarantee_min_count(file_lens[tst_train_classes].astype(float), tst_train_num_insts, 1)
                            tst_tst   = get_num_samples_and_guarantee_min_count(file_lens[tst_tst_classes].astype(float), tst_tst_num_insts, 1)
                            counts[dataset_type][setting]['test'][fold_num][tst_train_classes] = tst_train
                            counts[dataset_type][setting]['test'][fold_num][tst_tst_classes]   = tst_tst


                    # Few-shot settings.
                    else:
                        # TODO: In Generalised few-shot learning we want all instances to have at least num shots during training!!!!!
                        n = int(setting[3 if setting[:2] == 'FS' else 4:])
                        counts[dataset_type][setting]['train'][fold_num][classes[trn_clss:]] += n

                        train_classes = classes[:trn_clss]
                        done_num_insts = counts[dataset_type][setting]['train'][fold_num].sum()
#                         trn = get_num_samples_and_guarantee_min_count(file_lens[train_classes].astype(float), num_insts[dataset_type]['train'] - done_num_insts, n)
#                         counts[dataset_type][setting]['train'][fold_num][train_classes] = trn

                        if (setting[:2] == 'FS'):
                            trn = get_num_samples_and_guarantee_min_count(file_lens[train_classes].astype(float), num_insts[dataset_type]['train'] - done_num_insts, 1)
                            counts[dataset_type][setting]['train'][fold_num][train_classes] = trn
                        
                            val_classes = classes[trn_clss:trn_clss+val_clss]
                            val = get_num_samples_and_guarantee_min_count(file_lens[val_classes].astype(float), num_insts[dataset_type]['val'], 1)
                            counts[dataset_type][setting]['val'][fold_num][val_classes] = val

                            test_classes = classes[trn_clss+val_clss:]
                            tst = get_num_samples_and_guarantee_min_count(file_lens[test_classes].astype(float), num_insts[dataset_type]['test'], 1)
                            counts[dataset_type][setting]['test'][fold_num][test_classes] = tst

                        else:
                            trn = get_num_samples_and_guarantee_min_count(file_lens[train_classes].astype(float), num_insts[dataset_type]['train'] - done_num_insts, n)
                            counts[dataset_type][setting]['train'][fold_num][train_classes] = trn
                        
                            # *** Validation Split *** #
                            # First determine the number of instances that will belong to the seen classes and the number of instances that will belong to the unseen ones.
                            val_train_num_insts = int(generalised_settings_percent_seen_classes*num_insts[dataset_type]['val'])
                            val_val_num_insts   = num_insts[dataset_type]['val'] - val_train_num_insts

                            # Start by sampling the seen classes for the validation.
                            val_train_classes = classes[:trn_clss]
                            val_val_classes   = classes[trn_clss:trn_clss+val_clss]
                            val_train = get_num_samples_and_guarantee_min_count(file_lens[val_train_classes].astype(float), val_train_num_insts, 1)
                            val_val   = get_num_samples_and_guarantee_min_count(file_lens[val_val_classes].astype(float), val_val_num_insts, 1)
                            counts[dataset_type][setting]['val'][fold_num][val_train_classes] = val_train
                            counts[dataset_type][setting]['val'][fold_num][val_val_classes]   = val_val

                            # *** Test Split *** #
                            # First determine the number of instances that will belong to the seen classes and the number of instances that will belong to the unseen ones.
                            tst_train_num_insts = int(generalised_settings_percent_seen_classes*num_insts[dataset_type]['test'])
                            tst_tst_num_insts   = num_insts[dataset_type]['test'] - tst_train_num_insts

                            # Start by sampling the seen classes for the validation.
                            tst_train_classes = classes[:trn_clss]
                            tst_tst_classes   = classes[trn_clss+val_clss:]
                            tst_train = get_num_samples_and_guarantee_min_count(file_lens[tst_train_classes].astype(float), tst_train_num_insts, 1)
                            tst_tst   = get_num_samples_and_guarantee_min_count(file_lens[tst_tst_classes].astype(float), tst_tst_num_insts, 1)
                            counts[dataset_type][setting]['test'][fold_num][tst_train_classes] = tst_train
                            counts[dataset_type][setting]['test'][fold_num][tst_tst_classes]   = tst_tst


                    classes_in_split[dataset_type][setting][fold_num]['train'] = train_classes

                    if (val_classes is not None):
                        classes_in_split[dataset_type][setting][fold_num]['val'] = val_classes
                    else:
                        classes_in_split[dataset_type][setting][fold_num]['val'] = {}
                        classes_in_split[dataset_type][setting][fold_num]['val']['seen']   = val_train_classes
                        classes_in_split[dataset_type][setting][fold_num]['val']['unseen'] = val_val_classes

                    if (test_classes is not None):
                        classes_in_split[dataset_type][setting][fold_num]['test'] = test_classes
                    else:
                        classes_in_split[dataset_type][setting][fold_num]['test'] = {}
                        classes_in_split[dataset_type][setting][fold_num]['test']['seen']   = tst_train_classes
                        classes_in_split[dataset_type][setting][fold_num]['test']['unseen'] = tst_tst_classes

                    num_instances_in_fold = np.copy(counts[dataset_type][setting]['train'][fold_num])
                    num_instances_in_fold += counts[dataset_type][setting]['val'][fold_num]
                    num_instances_in_fold += counts[dataset_type][setting]['test'][fold_num]
                    assert num_instances_in_fold.sum() == sum(num_insts[dataset_type][split] for split in splits)
                    assert (num_instances_in_fold != 0).all()




##############################################################################################################################
# First we take care of guaranteeing that the Hyperparemeter Tuning sets can be formed, according the chosen specifications. #
##############################################################################################################################
        err_msg = "No solution was found, for these settings, as some classes require more observations than the ones that exist."
        total_counts_HT = np.zeros(len(relation_filepointers)).astype(np.int64)

        if (disjoint_settings_HT):
            if (completely_disjoint_folds):
                for setting in evaluation_settings:
                    setting_counts = np.zeros(len(relation_filepointers)).astype(np.int64)
                    for fold_num in range(K_folds['hyperparameter']):
                        fold_counts = np.copy(counts['hyperparameter'][setting]['train'][fold_num])
                        fold_counts += counts['hyperparameter'][setting]['val'][fold_num]
                        fold_counts += counts['hyperparameter'][setting]['test'][fold_num]
                        assert np.less_equal(fold_counts, file_lens).all(), err_msg
                        setting_counts += fold_counts
                    assert np.less_equal(setting_counts, file_lens).all(), err_msg + " HYPERPARAMETER" + " Setting = " + setting
                    total_counts_HT += setting_counts
                assert np.less_equal(total_counts_HT, file_lens).all(), err_msg + " HYPERPARAMETER" + " Total Disjointness"

                
            if (disjoint_train_and_val_splits):
                maximums_test = np.zeros(len(relation_filepointers)).astype(np.int64)
                total_train_val_counts_HT = np.zeros(len(relation_filepointers)).astype(np.int64)
                for setting in evaluation_settings:
                    maximums_test_setting = np.zeros(len(relation_filepointers)).astype(np.int64)
                    setting_train_val_counts = np.zeros(len(relation_filepointers)).astype(np.int64)
                    for fold_num in range(K_folds['hyperparameter']):
                        fold_train_val_counts = np.copy(counts['hyperparameter'][setting]['train'][fold_num])
                        fold_train_val_counts += counts['hyperparameter'][setting]['val'][fold_num]
                        maximums_test_setting = np.maximum(maximums_test_setting, counts['hyperparameter'][setting]['test'][fold_num])
                        maximums_test = np.maximum(maximums_test, counts['hyperparameter'][setting]['test'][fold_num])
                        assert np.less_equal(fold_train_val_counts + counts['hyperparameter'][setting]['test'][fold_num], file_lens).all(), err_msg
                        setting_train_val_counts += fold_train_val_counts
                    setting_counts = setting_train_val_counts + maximums_test_setting
                    setting_err_msg = err_msg + " HYPERPARAMETER" + " Setting = " + setting
                    assert np.less_equal(setting_counts, file_lens).all(), setting_err_msg
                    total_train_val_counts_HT += setting_train_val_counts
                total_counts_HT = total_train_val_counts_HT + maximums_test
                assert np.less_equal(total_counts_HT, file_lens).all()

        # No disjoint settings in HT.
        else:
            if (completely_disjoint_folds):
                for setting in evaluation_settings:
                    setting_counts = np.zeros(len(relation_filepointers)).astype(np.int64)
                    for fold_num in range(K_folds['hyperparameter']):
                        fold_counts = np.copy(counts['hyperparameter'][setting]['train'][fold_num])
                        fold_counts += counts['hyperparameter'][setting]['val'][fold_num]
                        fold_counts += counts['hyperparameter'][setting]['test'][fold_num]
                        assert np.less_equal(fold_counts, file_lens).all(), err_msg
                        setting_counts += fold_counts
                    assert np.less_equal(setting_counts, file_lens).all(), err_msg + " HYPERPARAMETER" + " Setting = " + setting
                    total_counts_HT = np.maximum(total_counts_HT, setting_counts)
                assert np.less_equal(total_counts_HT, file_lens).all(), err_msg + " HYPERPARAMETER" + " Total Disjointness"

                
            if (disjoint_train_and_val_splits):
                for setting in evaluation_settings:
                    maximums_test_setting = np.zeros(len(relation_filepointers)).astype(np.int64)
                    setting_train_val_counts = np.zeros(len(relation_filepointers)).astype(np.int64)
                    for fold_num in range(K_folds['hyperparameter']):
                        fold_train_val_counts = np.copy(counts['hyperparameter'][setting]['train'][fold_num])
                        fold_train_val_counts += counts['hyperparameter'][setting]['val'][fold_num]
                        assert np.less_equal(fold_train_val_counts + counts['hyperparameter'][setting]['test'][fold_num], file_lens).all(), err_msg
                        maximums_test_setting = np.maximum(maximums_test_setting, counts['hyperparameter'][setting]['test'][fold_num])
                        setting_train_val_counts += fold_train_val_counts
                    setting_counts = setting_train_val_counts + maximums_test_setting
                    setting_err_msg = err_msg + " HYPERPARAMETER" + " Setting = " + setting
                    assert np.less_equal(setting_counts, file_lens).all(), setting_err_msg
                    total_counts_HT = np.maximum(total_counts_HT, setting_counts)
                assert np.less_equal(total_counts_HT, file_lens).all()

##########################################################################################################################
# Now we take care of guaranteeing that the Final Evaluation sets can be formed, according the chosen specifications and #
# having the val & test splits being disjoint from the HT sets, between themselves and disjoint with the training set.   #
##########################################################################################################################

        err_msg += " FINAL EVALUATION"
        if (completely_disjoint_folds):
            for setting in evaluation_settings:
                setting_counts = np.zeros(len(relation_filepointers)).astype(np.int64)
                train_counts = np.zeros(len(relation_filepointers)).astype(np.int64)
                val_test_counts = np.zeros(len(relation_filepointers)).astype(np.int64)
                for fold_num in range(K_folds['final']):
                    fold_counts = np.copy(counts['final'][setting]['train'][fold_num])
                    train_counts += fold_counts
                    fold_counts += counts['final'][setting]['val'][fold_num]
                    fold_counts += counts['final'][setting]['test'][fold_num]
                    setting_counts += fold_counts
                    assert np.less_equal(fold_counts, file_lens).all(), err_msg
                assert np.less_equal(setting_counts, file_lens).all(), err_msg + " Setting = " + setting
                assert np.less_equal(np.maximum(total_counts_HT, train_counts) + val_test_counts, file_lens).all(), err_msg + " Setting = " + setting


        if (disjoint_train_and_val_splits):
            for setting in evaluation_settings:
                setting_counts = np.zeros(len(relation_filepointers)).astype(np.int64)
                train_counts = np.zeros(len(relation_filepointers)).astype(np.int64)
                val_counts = np.zeros(len(relation_filepointers)).astype(np.int64)
                maximums_test_setting = np.zeros(len(relation_filepointers)).astype(np.int64)
                for fold_num in range(K_folds['final']):
                    fold_counts = np.copy(counts['final'][setting]['train'][fold_num])
                    train_counts += fold_counts
                    fold_counts += counts['final'][setting]['val'][fold_num]
                    val_counts += counts['final'][setting]['val'][fold_num]
                    fold_counts += counts['final'][setting]['test'][fold_num]
                    assert np.less_equal(fold_counts, file_lens).all(), err_msg
                    maximums_test_setting = np.maximum(maximums_test_setting, counts['final'][setting]['test'][fold_num])
                setting_counts = train_counts + val_counts + maximums_test_setting
                setting_err_msg = err_msg + " Setting = " + setting
                assert np.less_equal(setting_counts, file_lens).all(), setting_err_msg
                assert np.less_equal(np.maximum(total_counts_HT, train_counts) + val_counts + maximums_test_setting, file_lens).all(), setting_err_msg
    
        no_solution_yet = False
        print('\rCONFIGURATION FOUND AFTER', num_tries, 'ATTEMPTS.')
    except AssertionError as err:
        num_tries += 1
        if (num_tries % 25 == 0):
            print("\r", num_tries, end="", flush=True)
    #         print("\r", num_tries, err, end="", flush=True)

#     if (disjoint_train_and_val_splits):



# if (completely_disjoint_folds):
#     assert (zero_shot_class_counts_matrix.sum(axis=1) == train + val + test).all()
#     assert (zero_shot_class_counts_matrix != 0).all()
# if (only_disjoint_train_splits or disjoint_train_and_val_splits):
#     assert ((zero_shot_class_counts_matrix_train + zero_shot_class_counts_matrix_val + zero_shot_class_counts_matrix_test).sum(axis=1) == train + val + test).all()
#     assert (zero_shot_class_counts_matrix_train + zero_shot_class_counts_matrix_val + zero_shot_class_counts_matrix_test != 0).all()

# err_msg = "It is impossible to create folds with this distribution, as some classes require more observations than the ones that exist."
# if (completely_disjoint_folds):
#     assert np.less_equal(zero_shot_class_counts_matrix.sum(axis=0), file_lens).all(), err_msg
# if (only_disjoint_train_splits):
#     train_sum = zero_shot_class_counts_matrix_train.sum(axis=0)
#     assert all(np.less_equal(train_sum + zero_shot_class_counts_matrix_val[fold_num] + zero_shot_class_counts_matrix_test[fold_num], file_lens).all() for fold_num in range(K_folds)), err_msg
# if (disjoint_train_and_val_splits):
#     train_val_sum = (zero_shot_class_counts_matrix_train + zero_shot_class_counts_matrix_val).sum(axis=0)
#     assert all(np.less_equal(train_val_sum + zero_shot_class_counts_matrix_test[fold_num], file_lens).all() for fold_num in range(K_folds)), err_msg










# experiment_types = ['normal', 'zero_shot', 'few_shot']
# dataset_types = ['train', 'val', 'test']
# new_data_path = path_datasets + 'Ours/sentences_only_test/'

# relations_in_split_file_pointers = {}
# for e_t_num, experiment_type in enumerate(experiment_types):
#     # Make sure the experiment_type Directory exists
#     directory = os.path.dirname(new_data_path + experiment_type + '/')
#     if (not os.path.exists(directory)):
#         os.makedirs(directory)

#     relations_in_split_file_pointers[experiment_type] = {}
#     for d_t_num, dataset_type in enumerate(dataset_types):
#         file_name = new_data_path + experiment_type + '/' + dataset_type + '_relations.txt'
#         relations_in_split_file_pointers[experiment_type][dataset_type] = open(file_name, 'w', encoding='utf-8')

# for r_num, relation in enumerate(relation_filepointers):

#     file_name_positive = path_datasets + 'Levy/by_relation/positive_' + str(max_x_len) + '/' + re.sub('/', '-', re.sub(' ', '_', relation)) + '.txt'

#     file_length = file_len(file_name_positive)
#     data = []
#     with open(file_name_positive, "r", encoding='utf-8') as f:
#         for l_num, line in enumerate(f):
#             if (l_num % min(int(file_length/3), 250) == 0 or l_num + 1 == file_length): #TODO test efficiency of order in boolean tests with timeit
#                 print('\r' + str(r_num+1) + '/' + str(len(relation_filepointers)) + ' (' + relation + ')' +
#                       ' ||| ' + str(l_num + 1) + '/' + str(file_length), end="", flush=True)

#             line_temp = line.rstrip().split("\t")

#             line_temp_tokenized = space_word_tokenize_string(line_temp[0], tokenizer)

#             masked_sentence = []
#             for word in line_temp_tokenized:
#                 if (word[:7] == 'ENTITY_'):
#                     if (word == 'ENTITY_0'):
#                         masked_sentence.append("SUBJECT_ENTITY")
#                     else:
#                         masked_sentence.append("OBJECT_ENTITY")
#                 else:
#                     masked_sentence.append(word)
#             data.append(masked_sentence)

#     # ----------------- #
#     #   Normal setting  #
#     # ----------------- #
#     if (train+val+test > len(data)):
#         raise ValueError("Normal Setting - Attempted to extract too many distinct instances for class: " + relation + " (Asked for " + str(train+val+test) +
#                          " when only " + str(len(data)) + " are available).")

#     indices = list(range(0, len(data)))
#     random.shuffle(indices)
#     picked_indices = indices[0:train+val+test]
    
#     # Train set
#     for index in picked_indices[0:train]:
#         normal_setting[0].append(data[index])
#         normal_setting_labels[0].append(relation)

#     # Validation Set
#     for index in picked_indices[train:train+val]:
#         normal_setting[1].append(data[index])
#         normal_setting_labels[1].append(relation)

#     # Test Set
#     for index in picked_indices[train+val:]:
#         normal_setting[2].append(data[index])
#         normal_setting_labels[2].append(relation)


#     # Register relations in each split
#     for d_t_num, dataset_type in enumerate(dataset_types):
#         relations_in_split_file_pointers['normal'][dataset_type].write(relation + '\n')



#     # -------------------- #
#     #   Zero-Shot setting  #
#     # -------------------- #
#     if (relation not in zero_shot_training_classes[zero_shot_training_classes_num:]):
#         num_indices = train
#         dataset_index = 0
#         relations_in_split_file_pointers['zero_shot']['train'].write(relation + '\n')
#     elif (relation in zero_shot_training_classes[zero_shot_training_classes_num:zero_shot_training_classes_num + zero_shot_val_test_classes_num]):
#         num_indices = val
#         dataset_index = 1
#         relations_in_split_file_pointers['zero_shot']['val'].write(relation + '\n')
#     else:
#         num_indices = test
#         dataset_index = 2
#         relations_in_split_file_pointers['zero_shot']['test'].write(relation + '\n')

#     if (num_indices > len(data)):
#         raise ValueError("Zero-Shot - Attempted to extract too many distinct instances for class: " + relation + " (Asked for " + str(num_indices) +
#                          " when only " + str(len(data)) + " are available).")
    
#     indices = list(range(0, len(data)))
#     random.shuffle(indices)
#     picked_indices = indices[:num_indices]


#     for index in picked_indices[:num_indices]:
#         zero_shot_setting[dataset_index].append(data[index])
#         zero_shot_setting_labels[dataset_index].append(relation)


#     # ------------------- #
#     #   Few-Shot setting  #
#     # ------------------- #
#     if (relation not in few_shot_training_classes[few_shot_training_classes_num:]):
#         num_indices = train
#     elif (relation in few_shot_training_classes[few_shot_training_classes_num:few_shot_training_classes_num + few_shot_val_test_classes_num]):
#         num_indices = train_val_few_shot + val
#     else:
#         num_indices = train_test_few_shot + test

#     if (num_indices > len(data)):
#         raise ValueError("Few-Shot - Attempted to extract too many distinct instances for class: " + relation + " (Asked for " + str(num_indices) +
#                          " when only " + str(len(data)) + " are available).")
#     indices = list(range(0, len(data)))
#     random.shuffle(indices)
#     picked_indices = indices[:num_indices]

#     # All relations exist in the train split.
#     relations_in_split_file_pointers['few_shot']['train'].write(relation + '\n')

#     if (relation not in few_shot_training_classes[few_shot_training_classes_num:]):
#         # Train Set - Exclusive training classes
#         for index in picked_indices[:train]:
#             few_shot_setting[0].append(data[index])
#             few_shot_setting_labels[0].append(relation)
#     elif (relation in few_shot_training_classes[few_shot_training_classes_num:few_shot_training_classes_num + few_shot_val_test_classes_num]):
#         relations_in_split_file_pointers['few_shot']['val'].write(relation + '\n')
#         # Train Set - Few shot examples from validation set
#         for index in picked_indices[:train_val_few_shot]:
#             few_shot_setting[0].append(data[index])
#             few_shot_setting_labels[0].append(relation)
#         # Validation Set
#         for index in picked_indices[train_val_few_shot:]:
#             few_shot_setting[1].append(data[index])
#             few_shot_setting_labels[1].append(relation)
#     else:
#         relations_in_split_file_pointers['few_shot']['test'].write(relation + '\n')
#         # Train Set - Few shot examples from test set
#         for index in picked_indices[:train_test_few_shot]:
#             few_shot_setting[0].append(data[index])
#             few_shot_setting_labels[0].append(relation)
#         # Test Set
#         for index in picked_indices[train_test_few_shot:]:
#             few_shot_setting[2].append(data[index])
#             few_shot_setting_labels[2].append(relation)


# # Close filepointers that register which relations are in each split
# for e_t_num, experiment_type in enumerate(experiment_types):
#     for d_t_num, dataset_type in enumerate(dataset_types):
#         relations_in_split_file_pointers[experiment_type][dataset_type].close()



# datasets = [(normal_setting, normal_setting_labels), (zero_shot_setting, zero_shot_setting_labels), (few_shot_setting, few_shot_setting_labels)]

# for e_t_num, experiment_type in enumerate(experiment_types):
#     for d_t_num, dataset_type in enumerate(dataset_types):
#         # Save new split for the corresponding experiment type
#         with open(new_data_path + experiment_type + '/' + dataset_type + '.txt', "w+", encoding='utf-8') as f:
#             for datapoint in datasets[e_t_num][0][d_t_num]:
#                 f.write(''.join(datapoint) + '\n')


#         #####################
#         # Create the unsupervised labels, by means of assigning ids to the tokens in the train splits. We only care about the ones in the train
#         # split because only here will we use them, as means of training the model in an unsupervised way.
#         if (dataset_type == 'train'):
#             idx_to_token_map = []
#             token_to_idx_map = {}
#             data = []
#             with open(new_data_path + experiment_type + '/' + dataset_type + '.txt', "r", encoding='utf-8') as f:
#                 for line in f:
#                     data.append(line.strip().split())
#                     for token in data[-1]:
#                         if (token not in token_to_idx_map):                 # Read from bottom up - Maybe fixed that issue, since I had to implement my own loss.
#                             idx_to_token_map.append(token)                  # The comment below is dumb as then we need to shift predictions in the model
#                             token_to_idx_map[token] = len(idx_to_token_map) # By not subtracting 1 we can reserve the 0 index for the <PAD> token

#             # Transform the tokenized sentences to indexed sentences
#             data_as_idxs = [[token_to_idx_map[token] for token in data_instance] for data_instance in data]

#             # Save indexed sentences, and corresponding maps, to memory. 'ulbs' stands for 'unsupervised labels'
#             with open(new_data_path + experiment_type + '/' + dataset_type + '.ulbs', "wb+") as f:
#                 pickle.dump(data_as_idxs, f)
#                 pickle.dump(idx_to_token_map, f)
#                 pickle.dump(token_to_idx_map, f)


#         #####################
#         # Save relation labels to file. 'lbs' stands for 'labels'
#         with open(new_data_path + experiment_type + '/' + dataset_type + '.lbs', "wb+") as f:
#             pickle.dump(datasets[e_t_num][1][d_t_num], f)

['N', 'ZS', 'GZS', 'FS-1', 'FS-2', 'FS-5', 'FS-10', 'GFS-1', 'GFS-2', 'GFS-5']
 158675

KeyboardInterrupt: 